In [1]:
import os
import pickle
import string

SAVEDICTIONARYTOPICKLEFILE = 1
USEPICKLEFILE = 1

REBUILDDICTIONARYFROMPICKLEFILE = 0

path = 'F:\LaTeX\LatexWork\Bible'
os.chdir(path)
print (os.getcwd())

import plotly

F:\LaTeX\LatexWork\Bible


In [2]:
# %load BibleWordList2.py
'''
Update 21 Deceber 2018, 0004
'''


import os
import pickle
import collections
import operator
import re

DEBUG = 0

SAVEDICTIONARYTOPICKLEFILE = 1
USEPICKLEFILE = 1
REBUILDDICTIONARYFROMPICKLEFILE = 0

punctuations = '''!()_;:,<>?@#$^.'''

exceptions = [
    'A', 'Accept', 'After', 'Aha', 'All','Alas','Also','Arise',
    'Amen', 'An', 'And', 'As', 'At', 'Awake','According',
    'Be', 'Because', 'Before', 'Behold', 'Blessed','Build','Better',
    'Blessings', 'Blind', 'Both', 'Bring', 'But','Beautiful',
    'By', 'Command', 'Concerning', 'Consider', 'Cut','Come',"Children's",
    'Deal', 'Deceit', 'Deep', 'Deliver', 'Depart','Did',
    'Doubtless', 'Each', 'Either', 'Even', 'Every','Excellent','Fear',
    'For', 'From', 'Get', 'Give', 'Go', 'Grace',
    'Great', 'Hatred', 'Having', 'He', 'Hear',
    'Hearing', 'Heaviness', 'Help', 'His', 'Hold',
    'Horror', 'How', 'If', 'In', 'Incline', 'Judge',
    'Leave', 'Let', 'Lo', 'Look', 'Lying', 'Lodge','Make','Must','Mark',
    'Many', 'Mine', 'My', 'Neither', 'Nevertheless','Nay',
    'Not', 'Now', 'Number', 'O', 'Of', 'On', 'One','Out',
    'Only', 'Open', 'Or', 'Order', 'Our', 'Pardon',
    'Plead', 'Princes', 'Quicken', 'Red', 'Remember',
    'Remove', 'Riches', 'Righteous', 'Rise', 'Rivers','Return',
    'Sacrifice', 'Salvation', 'Say', 'Seven', 'Shall','Spake',
    'So', 'Speak', 'Sprinkle', 'Stablish', 'Stand','Strong',
    'Surely', 'Take', 'Teach', 'That', 'The', 'Their',
    'Then', 'There', 'Therefore', 'These', 'They',
    'This', 'Those', 'Thou', 'Though', 'Through',
    'Thus', 'Thy', 'To', 'Treasures', 'Trouble','Told',
    'Turn', 'Two', 'Unless', 'Uphold', 'We',
    'What', 'Whatsoever', 'When', 'Where', 'Wherefore',
    'Wherewithal', 'Which', 'Who', 'Whom', 'Whoso',
    'Whosoever', 'Why', 'Wise', 'With', 'Withhold','Walk','Wisdom',
    'Would', 'Ye', 'Yea', 'Yet', 'Your']

properNameList = [
    'Aaron', 'Abelmizraim', 'Abiasaph', 'Abib', 'Abihu', 'Abraham', 'Abram',
    'Aceldama', 'Achaia', 'Achaicus', 'Achor', 'Achzib', 'Adam', 'Adramyttium',
    'Adria', 'Adullam', 'Agrippa', 'Ahab', 'Ahaz', 'Ahikam', 'Ahisamach',
    'Aholiab', 'Alexandria', 'Alexandrians', 'Alphaeus', 'Amalek', 'Amittai',
    'Ammi ', 'Amminadab', 'Amon', 'Amorite', 'Amorites', 'Amram', 'Anathoth',
    'Andrew', 'Angel', 'Antioch', 'Aphrah', 'Apollos', 'Apphia', 'Aquila',
    'Arabia ', 'Arabian', 'Archippus', 'Ard', 'Areli', 'Aretas', 'Aristarchus',
    'Arnon', 'Arodi', 'Aroer ', 'Asenath', 'Asenath', 'Ashbel', 'Asher',
    'Ashkelon', 'Asia', 'Assir', 'Assyria ', 'Assyrian', 'Atad', 'Augustus',
    'Azariah', 'Azur', 'BRANCH', 'Baal ', 'Baali ', 'Baalim', 'Baalzephon',
    'Bablyon', 'Balaam', 'Balak', 'Barbarian', 'Barjona', 'Barnabas',
    'Barsabas', 'Bartholomew', 'Baruch', 'Becher', 'Beeri', 'Beerlahairoi',
    'Beersheba', 'Belah', 'Belial', 'Belteshazzar', 'Benjamin', 'Beor',
    'Bered', 'Beriah', 'Bernice', 'Bethaven', 'Bethdiblathaim ', 'Bethel',
    'Bethezel', 'Bethgamul', 'Bethlehem', 'Bethmeon', 'Bethshemesh',
    'Bezaleel', 'Bilhah', 'Bozrah', 'Caesar ', 'Caesarea', 'Cainan', 'Cana',
    'Canaan', 'Canaanite', 'Canaanites', 'Canaanitish', 'Capernaum', 'Caphtor',
    'Carchemish', 'Carmi', 'Castor', 'Cephas', 'Chaldeans', 'Chaldeans',
    'Chemosh', 'Chittim', 'Chloe', 'Christ', 'Cilicia', 'Clauda', 'Cnidus',
    'Crete', 'Cretians', 'Crispus', 'Cyprus', 'Cyrenians', 'Cyrus',
    'Damascenes', 'Damascus', 'Dan', 'Daniel', 'Darius', 'David', 'Demas',
    'Diblaim', 'Dibon', 'Didymus', 'Dinah', 'Edom', 'Egypt', 'Egyptian',
    'Egyptians', 'Ehi', 'Elasah', 'Eleazar', 'Elias', 'Eliezer', 'Elim',
    'Elisheba', 'Elkanah', 'Elon', 'Elzaphan', 'Enoch', 'Enos', 'Epaphras',
    'Ephesus', 'Ephraim', 'Ephrath', 'Ephron', 'Er', 'Eri', 'Esaias', 'Esau',
    'Etham', 'Euphrates', 'Euroclydon', 'Ezbon', 'Father', 'Felix', 'Festus',
    'Fortunatus', 'Gad', 'Gadarenes', 'Gaius', 'Galatia', 'Galilee', 'Gath',
    'Gaza', 'Gedaliah', 'Gemariah', 'Gentiles', 'Gera', 'Gershom', 'Gershon',
    'Gibeah', 'Gibeon', 'Gilead', 'Gilgal', 'God', 'Gomer', 'Goshen',
    'Grecians', 'Greek', 'Guni', 'Hagar', 'Haggi', 'Ham', 'Hananiah', 'Hanoch',
    'Heber', 'Hebrew', 'Hebrews', "Hebrews'", 'Hebron', 'Heldai', 'Helem',
    'Hen', 'Heshbon', 'Hezekiah', 'Hezron', 'High', 'Hilkiah', 'Hinnom',
    'Hittite', 'Hittites', 'Hivite', 'Hivites', 'Horeb', 'Horonaim', 'Hosea',
    'Hoshaiah', 'Huppim', 'Hur', 'Hushim', 'I', 'Isaac', 'Ishi', 'Ishmael',
    'Ishmeelites', 'Ishuah', 'Israel', 'Issachar', 'Issachar', 'Isui', 'Italy',
    'Ithamar', 'Ithiel', 'Izhar', 'JEHOVAH', 'Jachin', 'Jacob', 'Jahleel',
    'Jahzeel', 'Jairus', 'Jakeh', 'James', 'Jamin', 'Japheth', 'Jareb:',
    'Jared', 'Jebusite', 'Jebusites', 'Jeconiah', 'Jedaiah', 'Jehoiada',
    'Jehoiakim', 'Jehoshapha', 'Jehovahnissi', 'Jehu', 'Jemuel', 'Jeremiah',
    'Jerusalem', 'Jesus', 'Jethro', 'Jews', 'Jezer', 'Jezreel', 'Jimnah',
    'Joash', 'Job', 'Jochebed', 'Joel', 'Johanan', 'John', 'Jonah', 'Jonas',
    'Joppa', 'Jordan', 'Josedech', 'Joseph', 'Josiah', 'Jotham', 'Judaea',
    'Judah', 'Judas', 'Julius', 'Just ', 'Justus ', 'Kadesh', 'Kareah',
    'Kedar', 'Kerioth', 'King', 'Kiriathaim ', 'Kohath', 'Korah', 'Korhites',
    'LORD', 'Laban', 'Laban', 'Lachis', 'Lamech', 'Laodicea', 'Lasea', 'Leah',
    'Lebanon', 'Legion', 'Levi', 'Libertines', 'Libni', 'Libyans', 'Loammi',
    'Lord', 'Lord Jesus Christ', 'Loruhamah', 'Lucas', 'Luhith', 'Luz', 'Lycia',
    'Lydians', 'Maaseiah', 'Macedonia', 'Macedonian', 'Machir', 'Machpelah',
    'Mahalaleel', 'Mahali', 'Malachi', 'Malchiel', 'Mamre', 'Manasseh',
    'Marcus', 'Mareshah', 'Maroth', 'Massah', 'Matthew', 'Matthias',
    'Melchizedek', 'Melita', 'Melzar', 'Merari', 'Meribah', 'Messias',
    'Methuselah', 'Micah', 'Midian', 'Migdol', 'Miriam', 'Misgab', 'Mishael',
    'Mizpah', 'Moab', 'Morasthite', 'Moreshethgath', 'Moses', 'Muppim', 'Mushi',
    'Myra', 'Naaman', 'Naashon', 'Nadab', 'Naphtali', 'Nathanael', 'Nazareth',
    'Nebo', 'Nebuchadnezzar', 'Nebuchadrezzar', 'Nebuzaradan', 'Nehelamite',
    'Nepheg', 'Neriah', 'Nicanor', 'Nicolas', 'Nicopolis', 'Nineveh', 'No ',
    'Noah', 'Noph', 'Ohad', 'Olivet', 'Omri', 'Onan', 'Onesimus', 'Padan',
    'Padanaram', 'Palestine', 'Pallu', 'Pamphylia', 'Parmenas', 'Pashur',
    'Pathros', 'Paul', 'Pentecost', 'Pergamos', 'Perizzite', 'Perizzites',
    'Peter', 'Pethuel', 'Phallu', 'Pharaoh', "Pharaoh's", 'Pharaohhophra ',
    'Pharaohnecho', 'Pharez', 'Pharisee ', 'Pharisees', 'Phenice',
    'Philadelphia', 'Philemon', 'Philip', 'Philippi', 'Philistines',
    'Phinehas', 'Phuvah', 'Pihahiroth', 'Pilate', 'Pithom', 'Pollux',
    'Potiphar', 'Potipherah', 'Prince', 'Priscilla', 'Prochorus', 'Puah',
    'Publius', 'Puteoli', 'Putiel', 'Raamses', 'Rachel', 'Ramah', 'Rameses',
    'Rephidim', 'Reuben', 'Reuel', 'Rhegium', 'Rock', 'Rome', 'Rosh', 'Ruhamah',
    'Sabeans', 'Sadducees', 'Salem', 'Salmone', 'Samaria', 'Samaritans',
    'Samuel', 'Saphir', 'Sara', 'Sarah', 'Sarai', 'Sardis', 'Satan', 'Saul',
    'Saviour', 'Scythian', 'Selah', 'Sepharad', 'Serah', 'Sered', 'Seth',
    'Shaphan', 'Shaul', 'Shelah', 'Shem', 'Shemaiah', 'Shillem', 'Shiloh',
    'Shimi', 'Shimron', 'Shiphrah', 'Shittim', 'Shuni', 'Sidon', 'Sihon',
    'Sihor', 'Simeon', 'Simon ', 'Simon Zelotes', 'Sinai', 'Sirion', 'Smyrna',
    'Solomon', 'Sosthenes', 'Stephanas', 'Stephen', 'Succoth', 'Sychar',
    'Syracuse', 'Tabor', 'Tahpanhes', 'Tarshish', 'Teman', 'Theophilus',
    'Thessalonica', 'Thomas', 'Thummim', 'Thyatira', 'Tiberias', 'Timon',
    'Timotheus', 'Timothy', 'Titus', 'Tobijah', 'Tola', 'Tophet', 'Tychicus',
    'Tyre', 'Tyrus', 'Ucal', 'Uri', 'Urim', 'Uzziah', 'Uzziel', 'Zaanan',
    'Zaphnathpaaneah', 'Zarephath', 'Zebedee', 'Zebulun', 'Zedekiah',
    'Zephaniah', 'Zerah', 'Zichri', 'Zidon', 'Zilpah', 'Zion', 'Ziphion',
    'Zipporah', 'Zithri', 'Zohar']

##########
def removePunc(input):
    no_punc = ''
    for char in input:
        if char not in punctuations:
            no_punc = no_punc + char
    return no_punc


##########
def generatedLatexFormattedText(dict):
    BUILDAWIP = 1
    BUILDNWIV = 1
    BUILDPNIP = 1
    BUILDNWIC = 1
    BUILDNLIW = 1
    BUILDBPIP = 1
    MINISTRY = 1

    topString = "\\footnote{\\textcolor[cmyk]{0.99998,1,0,0}{\\hyperlink{TOC}{Return to end " \
                "of Table of Contents.}}}"
    textColorString = "\\textcolor[cmyk]{0.99998,1,0,0}{"
    endString = "}"

    '''
    Print plain text with Latex color header:
        \textcolor[rgb]{0.00,0.00,1.00} == BLUE
        \textcolor[cmyk]{0.99998,1,0,0} == A DEEPER BLUE
    '''
    print('\n\n%%%%%%%%%%%%%%%%%%%%%%%%%%%')
    print('Chapter with Latex format\n%%%%%%%%%%%%%%%%%%%%%%%%%%%\n\n')

    linesWith13Words = []
    linesWith18Words = []
    for line in dict:
        booknm = line[0]
        chapnm = line[1]
        chapno = line[2]
        versno = line[3]
        versn2 = line[4]
        newline = line[5]
        if versn2 == '1':
            print(topString + textColorString + newline + endString)
        else:
            print('[' + versn2 + '] ' + textColorString + newline + endString)
        words = newline.split()
        wordLength = len(words)
        if wordLength == 13:
            linesWith13Words.append(line)
        if wordLength == 18:
            linesWith18Words.append(line)
            
        
    '''
    NOW print the verses with LaTeX index items:
        AWIP - All Words in Passage
        NWIV - NUmber of Words in Verse
        PNIP - Proper noun in passage
        ... more later
    '''
    print('\n\n%%%%%%%%%%%%%%%%%%%%%%%%%%%')
    print('Chapter with indexed items\n%%%%%%%%%%%%%%%%%%%%%%%%%%%\n\n')
    for line in dict:
        booknm = line[0]
        chapnm = line[1]
        chapno = line[2]
        versno = line[3]
        versn2 = line[4]
        verseRef = "!" + booknm + "!" + chapnm + " " + chapno + ":" + versno 
        line = line[5]
        if versn2 == '1':
            verseString = topString + textColorString + line + endString
        else:
            verseString = '[' + versn2 + '] ' + textColorString + line + endString
        #print(booknm,chapnm,chapno,versno,versn2) 
        '''
        NWIV
        '''
        words = line.split()
        numWords = len(words)
        if DEBUG == 1:
            print(numWords,'Words in verse',versn2)
        extraString="\\index[NWIV]{" + str(numWords) + verseRef + "}"
        '''
        AWIP - here, Passage = verse
        '''
        verseDict = {}
        wordString = ''
        for word in words:
            word = removePunc(word)
            if word not in verseDict:
                verseDict[word] = 1
                wordString = wordString + "\\index[AWIP]{" + word + verseRef + "}"
            else:
                verseDict[word] = verseDict[word] + 1
                wordString = wordString + "\\index[AWIP]{" + word + verseRef 
                wordString = wordString + " (" + str(verseDict[word]) + ")}"
        '''
        PNIP - Proper names in passage
        '''
        propernameStrng = ''
        for item in verseDict:
            if item in properNameList:
                if item not in exceptions:
                    propernameStrng = propernameStrng + "\\index[PNIP]{" + item + verseRef + "}"

        builtVerseString = verseString + extraString + wordString  + propernameStrng
        print(builtVerseString)
    return linesWith18Words, linesWith13Words

##########
def showWordsByFrequency(dictionary,arg):
    r = re.compile("([a-zA-Z]+)([0-9]+)")
    m = r.match(arg)

    print('\\subsection{Words by Frequency}')
    print ("\\scriptsize")
    print ("\\begin{center}")
    print ("\\begin{longtable}{l|r}")
    print ("\caption[Word Frequencies in "+m.group(1)+" "+str(int(m.group(2)))+"]{Word Frequencies in "+m.group(1)+" "+str(int(m.group(2)))+"} \label{table:WordsIn-"+m.group(1)+"-"+str(int(m.group(2)))+"} \\\ ")    
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Word}} & \multicolumn{1}{c|}{\\textbf{Frequency}} \\\ \\hline ")
    print ("\\endfirsthead")
    print (" ")
    print ("\\multicolumn{2}{c}")
    print ("{{\\bfseries \\tablename\ \\thetable{} -- continued from previous page}} \\\ ")
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Word}} & \multicolumn{1}{c|}{\\textbf{Frequency}} \\\ \\hline ")
    print ("\\endhead")
    print (" ")
    print ("\hline \multicolumn{2}{|r|}{{Continued if needed}} \\\ \\hline")
    print ("\\endfoot")
    print (" ")
    print ("\hline \hline")
    print ("\\endlastfoot")
    sortedDictionary = sorted(dictionary.items(), key=lambda x: x[1], reverse=True)
    for item in sortedDictionary:
        print (item[0], "&", item[1], "\\\ \hline")
    print ("\\end{longtable}")
    print ("\\end{center}")
    print ("\n\n")
    print ("\\normalsize")
    print ("\n\n")    

##########
def showWordsAlphabetically(dictionary,arg):
    r = re.compile("([a-zA-Z]+)([0-9]+)")
    m = r.match(arg)

    print('\\subsection{Words Alphabetically}')
    print ("\\scriptsize")
    print ("\\begin{center}")
    print ("\\begin{longtable}{l|r}")
    print ("\caption[Word Alphabetically in "+m.group(1)+" "+str(int(m.group(2)))+"]{Word Alphabetically in "+m.group(1)+" "+str(int(m.group(2)))+"} \label{table:WordsIn-"+m.group(1)+"-"+str(int(m.group(2)))+"} \\\ ")    
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Word}} & \multicolumn{1}{c|}{\\textbf{Frequency}} \\\ \\hline ")
    print ("\\endfirsthead")
    print (" ")
    print ("\\multicolumn{2}{c}")
    print ("{{\\bfseries \\tablename\ \\thetable{} -- continued from previous page}} \\\ ")
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Word}} & \multicolumn{1}{c|}{\\textbf{Frequency}} \\\ \\hline ")
    print ("\\endhead")
    print (" ")
    print ("\hline \multicolumn{2}{|r|}{{Continued if needed}} \\\ \\hline")
    print ("\\endfoot")
    print (" ")
    print ("\hline \hline")
    print ("\\endlastfoot")
    sortedDictionary = sorted(dictionary.items(), key=lambda x: x[0], reverse=False)
    for item in sortedDictionary:
        print (item[0], "&", item[1], "\\\ \hline")
    print ("\\end{longtable}")
    print ("\\end{center}")
    print ("\n\n")
    print ("\\normalsize")
    print ("\n\n")    

##########
def generateWordStatisticsForChapter(label,verses):
    '''
    Need to simplify this code! ONLY count number of words, number of unique 
    words, number of itallic words, number of unique itallic words
    
    verseDictionary, rest for each verse
    chapterDictionary
    '''
    global properNameList
    wordCount = 0
    emphasizedwordCount = 0
    wordDict = {}
    emphasizedwordDict = {}
    verseString = ''
    A = re.findall(r'\d+', label)
    A = " ".join(map(str, A))
    digit = int(A)

    word1 = " ".join(re.findall("[a-zA-Z]+", label))
    
    print('\n\n%%%%%%%%%%%%%%%%%%%%%%%%%%%')
    print('%%%%% Word Statistics\n%%%%%%%%%%%%%%%%%%%%%%%%%%\n\n')
    
    print('\\normalsize')

    verseStats = ["\\subsection{Chapter Word Statistics}",
                  "\n",
                  "%%%%%%%%%%",
                  "%%%%%%%%%%",
                  " ",
                  "\\begin{center}",
                  "\\begin{longtable}{l|c|c|c|c}",
                  "\\caption[Stats for "+word1+" "+str(digit) +"]{Stats for "+word1+" "+str(digit) +"} \\label{table:Stats for "+word1+" "+str(digit) +"} \\\ ",
                  "\\hline \multicolumn{1}{|c|}{\\textbf{Verse(s)}} & \multicolumn{1}{|c|}{\\textbf{Count}} & \multicolumn{1}{|c|}{\\textbf{Unique}} & \multicolumn{1}{|c|}{\\textbf{Italics}} & \multicolumn{1}{|c|}{\\textbf{Uniq Italic}}  \\\ \\hline ",
                  "\\endfirsthead",
                  " ",
                  "\multicolumn{5}{c}",
                  "{{\\bfseries \\tablename\ \\thetable{} -- continued from previous page}} \\\  ",
                  "\\hline \multicolumn{1}{|c|}{\\textbf{Verse(s)}} & \multicolumn{1}{|c|}{\\textbf{Count}} & \multicolumn{1}{|c|}{\\textbf{Unique}} & \multicolumn{1}{|c|}{\\textbf{Italics}} & \multicolumn{1}{|c|}{\\textbf{Uniq Italic}}  \\\ \\hline ",
                  "\\endhead",
                  " ",
                  "\\hline \\multicolumn{5}{|r|}{{Continued if needed}} \\\ \\hline",
                  "\\endfoot ",]

    '''
    This section prints JUST the Bible text in a LaTeX format with a textcolor encoding added
    Original Code
    '''
    chapterDict = {}
    chapwordWords = 0
    chapterUniqueWords = 0
    numberChapterUniqueItallics = 0
    numberChapterItallics = 0
    verseNo = 0
    for line in verses:
        verseNo += 1
        verseDict = {}
#        print(line)
        words = removePunc(line[5])
        wordcount = 0
        uniqueWords = 0        
        '''
        Process for verse
        '''
        for word in words.split():
            wordcount += 1
            if word not in verseDict:
                verseDict[word] = 1
                uniqueWords += 1
            else:
                verseDict[word] = verseDict[word] + 1
        numberUniqueItallics = 0
        numberItallics = 0
        for item in verseDict:
            if "\emph{" in item:
                numberUniqueItallics += 1
                numberItallics = numberItallics + verseDict[item]                
        '''
        Process for chapter
        '''                
        for word in words.split():
            chapwordWords += 1
            if word not in chapterDict:
                chapterDict[word] = 1
                chapterUniqueWords += 1
            else:
                chapterDict[word] = chapterDict[word] + 1
        sequence = str(verseNo),str(wordcount),str(uniqueWords),str(numberItallics),str(numberUniqueItallics)+'\\\ \hline'
        S = ' & '.join(sequence)
        verseStats.append(S)
    for item in chapterDict:
        if "\emph{" in item:
            numberChapterUniqueItallics +=1
            numberChapterItallics = numberChapterItallics + chapterDict[item]

    '''
    Finish Up!
    '''
#    print ("\n",wordCount,"words in passage, of which",len(wordDict),"are unique")
#    print (emphasizedwordCount,'words in ital, of which',len(emphasizedwordDict),'are unique')
    print ("\n\n")
    verseStats.append("\hline \hline")
    sequence = 'Total',str(chapwordWords),str(chapterUniqueWords),str(numberChapterItallics),str(numberChapterUniqueItallics)
    S = ' & '.join(sequence)
    verseStats.append(S)
    for item in verseStats:
        print (item)
    print ("\n\n")
    print ("\\end{longtable}")
    print ("\\end{center}\n")
#    print (chapwordWords,"words in passage, of which",chapterUniqueWords,"are unique")
#    print (numberChapterItallics,'words in ital, of which',numberChapterUniqueItallics,'are unique')
#    print ("\n\n")
#
    print ("%%%%%%%%%%\n%%%%%%%%%%")
    print (" ")
    
    return chapterDict

##########
def getWordLengthsInChapter(dictionary,arg):
    r = re.compile("([a-zA-Z]+)([0-9]+)")
    m = r.match(arg)

    wordLengthDictionary = {}
    for item in dictionary:
        if "\emph{" in item:
            tempitem = item.replace("\emph{",'')
            tempitem = tempitem.replace('}','')
            if len(tempitem) in wordLengthDictionary:
                wordLengthDictionary[len(tempitem)] = wordLengthDictionary[len(tempitem)] + ', ' + item
            else:
                    wordLengthDictionary[len(tempitem)] = item
        else:
            if len(item) in wordLengthDictionary:
                wordLengthDictionary[len(item)] = wordLengthDictionary[len(item)] + ', ' + item
            else:
                wordLengthDictionary[len(item)] = item
    wordLengthDictionary = sorted(wordLengthDictionary.items(), key=lambda x:x[0], reverse=False)

    print('\\subsection{Word Lengths in Chapter}')
    print ("\\normalsize")
#    print ("\\begin{center}")
    print ("\\begin{longtable}{l|p{3.75in}}")
    print ("\caption[Words by Length in "+m.group(1)+" "+str(int(m.group(2)))+"]{Words by Length in "+m.group(1)+" "+str(int(m.group(2)))+"} \label{table:WordsIn-"+m.group(1)+"-"+str(int(m.group(2)))+"} \\\ ")    
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Length}} & \multicolumn{1}{c|}{\\textbf{Words}} \\\ \\hline ")
    print ("\\endfirsthead")
    print (" ")
    print ("\\multicolumn{2}{c}")
    print ("{{\\bfseries \\tablename\ \\thetable{} -- continued from previous page}} \\\ ")
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Length}} & \multicolumn{1}{c|}{\\textbf{Words}} \\\ \\hline ")
    print ("\\endhead")
    print (" ")
    print ("\hline \multicolumn{2}{|r|}{{Continued if needed}} \\\ \\hline")
    print ("\\endfoot")
    print (" ")
    print ("\hline \hline")
    print ("\\endlastfoot")
    for item in wordLengthDictionary:
        print (item[0], "&", item[1], "\\\ \hline")
    print ("\\end{longtable}")
#    print ("\\end{center}")
    print ("\n\n")
    print ("\n\n") 
    print ("%%%%%%%%%%\n%%%%%%%%%%")
    print (" ")

##########
def makeVersesWith13WordsTable(list,arg):
    print ("\n\n") 
    print ("%%%%%%%%%%\n%%%%%%%%%%")

    r = re.compile("([a-zA-Z]+)([0-9]+)")
    m = r.match(arg)


    print('\\subsection{Verses with 13 Words in Chapter}')
    print ("\\normalsize")
#    print ("\\begin{center}")
    print ("\\begin{longtable}{l|p{3.75in}}")
    print ("\caption[Verses with 13 Words  in "+m.group(1)+" "+str(int(m.group(2)))+"]{Verses with 13 Words  in "+m.group(1)+" "+str(int(m.group(2)))+"} \label{table:Verses with 13 Words in-"+m.group(1)+"-"+str(int(m.group(2)))+"} \\\ ")    
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Reference}} & \multicolumn{1}{c|}{\\textbf{Verse}} \\\ \\hline ")
    print ("\\endfirsthead")
    print (" ")
    print ("\\multicolumn{2}{c}")
    print ("{{\\bfseries \\tablename\ \\thetable{} -- continued from previous page}} \\\ ")
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Reference}} & \multicolumn{1}{c|}{\\textbf{Verse}} \\\ \\hline ")
    print ("\\endhead")
    print (" ")
    print ("\hline \multicolumn{2}{|r|}{{Continued if needed}} \\\ \\hline")
    print ("\\endfoot")
    print (" ")
    print ("\hline \hline")
    print ("\\endlastfoot")
    for item in list:
        print (item[0]+' '+item[2]+":"+item[4], "&", item[5], "\\\ \hline")
    print ("\\end{longtable}")
#    print ("\\end{center}")
    print ("\n\n")
    print ("\n\n") 
    print ("%%%%%%%%%%\n%%%%%%%%%%")
    print (" ")
        
##########
          
##########
def makeVersesWith18WordsTable(list,arg):
    print ("\n\n") 
    print ("%%%%%%%%%%\n%%%%%%%%%%")

    r = re.compile("([a-zA-Z]+)([0-9]+)")
    m = r.match(arg)


    print('\\subsection{Verses with 18 Words in Chapter}')
    print ("\\normalsize")
#    print ("\\begin{center}")
    print ("\\begin{longtable}{l|p{3.75in}}")
    print ("\caption[Verses with 18 Words  in "+m.group(1)+" "+str(int(m.group(2)))+"]{Verses with 18 Words  in "+m.group(1)+" "+str(int(m.group(2)))+"} \label{table:Verses with 18 Words in-"+m.group(1)+"-"+str(int(m.group(2)))+"} \\\ ")    
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Reference}} & \multicolumn{1}{c|}{\\textbf{Verse}} \\\ \\hline ")
    print ("\\endfirsthead")
    print (" ")
    print ("\\multicolumn{2}{c}")
    print ("{{\\bfseries \\tablename\ \\thetable{} -- continued from previous page}} \\\ ")
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Reference}} & \multicolumn{1}{c|}{\\textbf{Verse}} \\\ \\hline ")
    print ("\\endhead")
    print (" ")
    print ("\hline \multicolumn{2}{|r|}{{Continued if needed}} \\\ \\hline")
    print ("\\endfoot")
    print (" ")
    print ("\hline \hline")
    print ("\\endlastfoot")
    for item in list:
        print (item[0]+' '+item[2]+":"+item[4], "&", item[5], "\\\ \hline")
    print ("\\end{longtable}")
#    print ("\\end{center}")
    print ("\n\n")
    print ("\n\n") 
    print ("%%%%%%%%%%\n%%%%%%%%%%")
    print (" ")
        
##########

def countLettersInVerses(arg1,arg2):
    for item in arg2:
        letterDict = {}
        for item2 in item[5]:
            if "\emph" in item2:
                item2 = item2.replace('\emph','')
            if item2 != ' ':
                if item2 in letterDict:
                    letterDict[item2] = letterDict[item2] + 1
                else:
                    letterDict[item2] =  1
            print ("for verse",item[2],":",letterDict)

def ProcessVerses(arg1,arg2):
    ''' 
    arg1 is dictionary name and arg2 is the dictionary
    '''
#    saveToPickleFile(arg1,arg2)  
    linesWith18words, linesWith13words = generatedLatexFormattedText(arg2)         # format chapter in Latex
    #linesWith13words is for use later ... 
    chapterDict = generateWordStatisticsForChapter(arg1,arg2)   # do chapter word statistics
    showWordsByFrequency(chapterDict,arg1)                      # show words by frequency
    showWordsAlphabetically(chapterDict,arg1)                   # show words alphabetically
    getWordLengthsInChapter(chapterDict,arg1)                   # show word lengths in chapter
    
    if len(linesWith13words) > 0:
        #print(linesWith13words)
        makeVersesWith13WordsTable(linesWith13words,arg1)
        
    if len(linesWith18words) > 0:
        #print(linesWith13words)
        makeVersesWith18WordsTable(linesWith18words,arg1)
        
    ''' In Development
    countLettersInVerses(arg1,arg2)
    '''



In [3]:
path = 'F:\python'
os.chdir(path)
print (os.getcwd())

F:\python


In [4]:
# %load NEW07OT-MakeConcordance-Judges.py
"""
Created on Sun Jan 01 07:52:18 2017

@author: keith
"""


import os
import pickle
import string

from BibleWordList2 import *

BUILDAWIP = 1
BUILDNWIV = 1
BUILDPNIP = 1
BUILDNWIC = 1
BUILDNLIW = 1
BUILDBPIP = 1

SAVEDICTIONARYTOPICKLEFILE = 1
USEPICKLEFILE = 1
REBUILDDICTIONARYFROMPICKLEFILE = 0

Judges01 = [
    ['Judges','Jdg','01','001','1',"Now after the death of Joshua it came to pass, that the children of Israel asked the LORD, saying, Who shall go up for us against the Canaanites first, to fight against them?"],
['Judges','Jdg','01','002','2',"And the LORD said, Judah shall go up: behold, I have delivered the land into his hand."],
['Judges','Jdg','01','003','3',"And Judah said unto Simeon his brother, Come up with me into my lot, that we may fight against the Canaanites; and I likewise will go with thee into thy lot. So Simeon went with him."],
['Judges','Jdg','01','004','4',"And Judah went up; and the LORD delivered the Canaanites and the Perizzites into their hand: and they slew of them in Bezek ten thousand men."],
['Judges','Jdg','01','005','5',"And they found Adoni-bezek in Bezek: and they fought against him, and they slew the Canaanites and the Perizzites."],
['Judges','Jdg','01','006','6',"But Adoni-bezek fled; and they pursued after him, and caught him, and cut off his thumbs and his great toes."],
['Judges','Jdg','01','007','7',"And Adoni-bezek said, Threescore and ten kings, having their thumbs and their great toes cut off, gathered \emph{their} \emph{meat} under my table: as I have done, so God hath requited me. And they brought him to Jerusalem, and there he died."],
['Judges','Jdg','01','008','8',"Now the children of Judah had fought against Jerusalem, and had taken it, and smitten it with the edge of the sword, and set the city on fire."],
['Judges','Jdg','01','009','9',"And afterward the children of Judah went down to fight against the Canaanites, that dwelt in the mountain, and in the south, and in the valley."],
['Judges','Jdg','01','010','10',"And Judah went against the Canaanites that dwelt in Hebron: (now the name of Hebron before \emph{was} Kirjath-arba:) and they slew Sheshai, and Ahiman, and Talmai."],
['Judges','Jdg','01','011','11',"And from thence he went against the inhabitants of Debir: and the name of Debir before \emph{was} Kirjath-sepher:"],
['Judges','Jdg','01','012','12',"And Caleb said, He that smiteth Kirjath-sepher, and taketh it, to him will I give Achsah my daughter to wife."],
['Judges','Jdg','01','013','13',"And Othniel the son of Kenaz, Caleb's younger brother, took it: and he gave him Achsah his daughter to wife."],
['Judges','Jdg','01','014','14',"And it came to pass, when she came \emph{to} \emph{him}, that she moved him to ask of her father a field: and she lighted from off \emph{her} ass; and Caleb said unto her, What wilt thou?"],
['Judges','Jdg','01','015','15',"And she said unto him, Give me a blessing: for thou hast given me a south land; give me also springs of water. And Caleb gave her the upper springs and the nether springs."],
['Judges','Jdg','01','016','16',"And the children of the Kenite, Moses' father in law, went up out of the city of palm trees with the children of Judah into the wilderness of Judah, which \emph{lieth} in the south of Arad; and they went and dwelt among the people."],
['Judges','Jdg','01','017','17',"And Judah went with Simeon his brother, and they slew the Canaanites that inhabited Zephath, and utterly destroyed it. And the name of the city was called Hormah."],
['Judges','Jdg','01','018','18',"Also Judah took Gaza with the coast thereof, and Askelon with the coast thereof, and Ekron with the coast thereof."],
['Judges','Jdg','01','019','19',"And the LORD was with Judah; and he drave out \emph{the} \emph{inhabitants} \emph{of} the mountain; but could not drive out the inhabitants of the valley, because they had chariots of iron."],
['Judges','Jdg','01','020','20',"And they gave Hebron unto Caleb, as Moses said: and he expelled thence the three sons of Anak."],
['Judges','Jdg','01','021','21',"And the children of Benjamin did not drive out the Jebusites that inhabited Jerusalem; but the Jebusites dwell with the children of Benjamin in Jerusalem unto this day."],
['Judges','Jdg','01','022','22',"And the house of Joseph, they also went up against Beth-el: and the LORD \emph{was} with them."],
['Judges','Jdg','01','023','23',"And the house of Joseph sent to descry Beth-el. (Now the name of the city before \emph{was} Luz.)"],
['Judges','Jdg','01','024','24',"And the spies saw a man come forth out of the city, and they said unto him, Shew us, we pray thee, the entrance into the city, and we will shew thee mercy."],
['Judges','Jdg','01','025','25',"And when he shewed them the entrance into the city, they smote the city with the edge of the sword; but they let go the man and all his family."],
['Judges','Jdg','01','026','26',"And the man went into the land of the Hittites, and built a city, and called the name thereof Luz: which \emph{is} the name thereof unto this day."],
['Judges','Jdg','01','027','27',"Neither did Manasseh drive out \emph{the} \emph{inhabitants} \emph{of} Beth-shean and her towns, nor Taanach and her towns, nor the inhabitants of Dor and her towns, nor the inhabitants of Ibleam and her towns, nor the inhabitants of Megiddo and her towns: but the Canaanites would dwell in that land."],
['Judges','Jdg','01','028','28',"And it came to pass, when Israel was strong, that they put the Canaanites to tribute, and did not utterly drive them out."],
['Judges','Jdg','01','029','29',"Neither did Ephraim drive out the Canaanites that dwelt in Gezer; but the Canaanites dwelt in Gezer among them."],
['Judges','Jdg','01','030','30',"Neither did Zebulun drive out the inhabitants of Kitron, nor the inhabitants of Nahalol; but the Canaanites dwelt among them, and became tributaries."],
['Judges','Jdg','01','031','31',"Neither did Asher drive out the inhabitants of Accho, nor the inhabitants of Zidon, nor of Ahlab, nor of Achzib, nor of Helbah, nor of Aphik, nor of Rehob:"],
['Judges','Jdg','01','032','32',"But the Asherites dwelt among the Canaanites, the inhabitants of the land: for they did not drive them out."],
['Judges','Jdg','01','033','33',"Neither did Naphtali drive out the inhabitants of Beth-shemesh, nor the inhabitants of Beth-anath; but he dwelt among the Canaanites, the inhabitants of the land: nevertheless the inhabitants of Beth-shemesh and of Beth-anath became tributaries unto them."],
['Judges','Jdg','01','034','34',"And the Amorites forced the children of Dan into the mountain: for they would not suffer them to come down to the valley:"],
['Judges','Jdg','01','035','35',"But the Amorites would dwell in mount Heres in Aijalon, and in Shaalbim: yet the hand of the house of Joseph prevailed, so that they became tributaries."],
['Judges','Jdg','01','036','36',"And the coast of the Amorites \emph{was} from the going up to Akrabbim, from the rock, and upward."]
]

Judges02 = [
    ['Judges','Jdg','02','001','1',"And an angel of the LORD came up from Gilgal to Bochim, and said, I made you to go up out of Egypt, and have brought you unto the land which I sware unto your fathers; and I said, I will never break my covenant with you."],
['Judges','Jdg','02','002','2',"And ye shall make no league with the inhabitants of this land; ye shall throw down their altars: but ye have not obeyed my voice: why have ye done this?"],
['Judges','Jdg','02','003','3',"Wherefore I also said, I will not drive them out from before you; but they shall be \emph{as} \emph{thorns} in your sides, and their gods shall be a snare unto you."],
['Judges','Jdg','02','004','4',"And it came to pass, when the angel of the LORD spake these words unto all the children of Israel, that the people lifted up their voice, and wept."],
['Judges','Jdg','02','005','5',"And they called the name of that place Bochim: and they sacrificed there unto the LORD."],
['Judges','Jdg','02','006','6',"And when Joshua had let the people go, the children of Israel went every man unto his inheritance to possess the land."],
['Judges','Jdg','02','007','7',"And the people served the LORD all the days of Joshua, and all the days of the elders that outlived Joshua, who had seen all the great works of the LORD, that he did for Israel."],
['Judges','Jdg','02','008','8',"And Joshua the son of Nun, the servant of the LORD, died, \emph{being} an hundred and ten years old."],
['Judges','Jdg','02','009','9',"And they buried him in the border of his inheritance in Timnath-heres, in the mount of Ephraim, on the north side of the hill Gaash."],
['Judges','Jdg','02','010','10',"And also all that generation were gathered unto their fathers: and there arose another generation after them, which knew not the LORD, nor yet the works which he had done for Israel."],
['Judges','Jdg','02','011','11',"And the children of Israel did evil in the sight of the LORD, and served Baalim:"],
['Judges','Jdg','02','012','12',"And they forsook the LORD God of their fathers, which brought them out of the land of Egypt, and followed other gods, of the gods of the people that \emph{were} round about them, and bowed themselves unto them, and provoked the LORD to anger."],
['Judges','Jdg','02','013','13',"And they forsook the LORD, and served Baal and Ashtaroth."],
['Judges','Jdg','02','014','14',"And the anger of the LORD was hot against Israel, and he delivered them into the hands of spoilers that spoiled them, and he sold them into the hands of their enemies round about, so that they could not any longer stand before their enemies."],
['Judges','Jdg','02','015','15',"Whithersoever they went out, the hand of the LORD was against them for evil, as the LORD had said, and as the LORD had sworn unto them: and they were greatly distressed."],
['Judges','Jdg','02','016','16',"Nevertheless the LORD raised up judges, which delivered them out of the hand of those that spoiled them."],
['Judges','Jdg','02','017','17',"And yet they would not hearken unto their judges, but they went a whoring after other gods, and bowed themselves unto them: they turned quickly out of the way which their fathers walked in, obeying the commandments of the LORD; \emph{but} they did not so."],
['Judges','Jdg','02','018','18',"And when the LORD raised them up judges, then the LORD was with the judge, and delivered them out of the hand of their enemies all the days of the judge: for it repented the LORD because of their groanings by reason of them that oppressed them and vexed them."],
['Judges','Jdg','02','019','19',"And it came to pass, when the judge was dead, \emph{that} they returned, and corrupted \emph{themselves} more than their fathers, in following other gods to serve them, and to bow down unto them; they ceased not from their own doings, nor from their stubborn way."],
['Judges','Jdg','02','020','20',"And the anger of the LORD was hot against Israel; and he said, Because that this people hath transgressed my covenant which I commanded their fathers, and have not hearkened unto my voice;"],
['Judges','Jdg','02','021','21',"I also will not henceforth drive out any from before them of the nations which Joshua left when he died:"],
['Judges','Jdg','02','022','22',"That through them I may prove Israel, whether they will keep the way of the LORD to walk therein, as their fathers did keep \emph{it}, or not."],
['Judges','Jdg','02','023','23',"Therefore the LORD left those nations, without driving them out hastily; neither delivered he them into the hand of Joshua."]]

Judges03 = [
    ['Judges','Jdg','03','001','1',"Now these \emph{are} the nations which the LORD left, to prove Israel by them, \emph{even} as many \emph{of} \emph{Israel} as had not known all the wars of Canaan;"],
['Judges','Jdg','03','002','2',"Only that the generations of the children of Israel might know, to teach them war, at the least such as before knew nothing thereof;"],
['Judges','Jdg','03','003','3',"\emph{Namely}, five lords of the Philistines, and all the Canaanites, and the Sidonians, and the Hivites that dwelt in mount Lebanon, from mount Baal-hermon unto the entering in of Hamath."],
['Judges','Jdg','03','004','4',"And they were to prove Israel by them, to know whether they would hearken unto the commandments of the LORD, which he commanded their fathers by the hand of Moses."],
['Judges','Jdg','03','005','5',"And the children of Israel dwelt among the Canaanites, Hittites, and Amorites, and Perizzites, and Hivites, and Jebusites:"],
['Judges','Jdg','03','006','6',"And they took their daughters to be their wives, and gave their daughters to their sons, and served their gods."],
['Judges','Jdg','03','007','7',"And the children of Israel did evil in the sight of the LORD, and forgat the LORD their God, and served Baalim and the groves."],
['Judges','Jdg','03','008','8',"Therefore the anger of the LORD was hot against Israel, and he sold them into the hand of Chushan-rishathaim king of Mesopotamia: and the children of Israel served Chushan-rishathaim eight years."],
['Judges','Jdg','03','009','9',"And when the children of Israel cried unto the LORD, the LORD raised up a deliverer to the children of Israel, who delivered them, \emph{even} Othniel the son of Kenaz, Caleb's younger brother."],
['Judges','Jdg','03','010','10',"And the Spirit of the LORD came upon him, and he judged Israel, and went out to war: and the LORD delivered Chushan-rishathaim king of Mesopotamia into his hand; and his hand prevailed against Chushan-rishathaim."],
['Judges','Jdg','03','011','11',"And the land had rest forty years. And Othniel the son of Kenaz died."],
['Judges','Jdg','03','012','12',"And the children of Israel did evil again in the sight of the LORD: and the LORD strengthened Eglon the king of Moab against Israel, because they had done evil in the sight of the LORD."],
['Judges','Jdg','03','013','13',"And he gathered unto him the children of Ammon and Amalek, and went and smote Israel, and possessed the city of palm trees."],
['Judges','Jdg','03','014','14',"So the children of Israel served Eglon the king of Moab eighteen years."],
['Judges','Jdg','03','015','15',"But when the children of Israel cried unto the LORD, the LORD raised them up a deliverer, Ehud the son of Gera, a Benjamite, a man lefthanded: and by him the children of Israel sent a present unto Eglon the king of Moab."],
['Judges','Jdg','03','016','16',"But Ehud made him a dagger which had two edges, of a cubit length; and he did gird it under his raiment upon his right thigh."],
['Judges','Jdg','03','017','17',"And he brought the present unto Eglon king of Moab: and Eglon \emph{was} a very fat man."],
['Judges','Jdg','03','018','18',"And when he had made an end to offer the present, he sent away the people that bare the present."],
['Judges','Jdg','03','019','19',"But he himself turned again from the quarries that \emph{were} by Gilgal, and said, I have a secret errand unto thee, O king: who said, Keep silence. And all that stood by him went out from him."],
['Judges','Jdg','03','020','20',"And Ehud came unto him; and he was sitting in a summer parlour, which he had for himself alone. And Ehud said, I have a message from God unto thee. And he arose out of \emph{his} seat."],
['Judges','Jdg','03','021','21',"And Ehud put forth his left hand, and took the dagger from his right thigh, and thrust it into his belly:"],
['Judges','Jdg','03','022','22',"And the haft also went in after the blade; and the fat closed upon the blade, so that he could not draw the dagger out of his belly; and the dirt came out."],
['Judges','Jdg','03','023','23',"Then Ehud went forth through the porch, and shut the doors of the parlour upon him, and locked them."],
['Judges','Jdg','03','024','24',"When he was gone out, his servants came; and when they saw that, behold, the doors of the parlour \emph{were} locked, they said, Surely he covereth his feet in his summer chamber."],
['Judges','Jdg','03','025','25',"And they tarried till they were ashamed: and, behold, he opened not the doors of the parlour; therefore they took a key, and opened \emph{them}: and, behold, their lord \emph{was} fallen down dead on the earth."],
['Judges','Jdg','03','026','26',"And Ehud escaped while they tarried, and passed beyond the quarries, and escaped unto Seirath."],
['Judges','Jdg','03','027','27',"And it came to pass, when he was come, that he blew a trumpet in the mountain of Ephraim, and the children of Israel went down with him from the mount, and he before them."],
['Judges','Jdg','03','028','28',"And he said unto them, Follow after me: for the LORD hath delivered your enemies the Moabites into your hand. And they went down after him, and took the fords of Jordan toward Moab, and suffered not a man to pass over."],
['Judges','Jdg','03','029','29',"And they slew of Moab at that time about ten thousand men, all lusty, and all men of valour; and there escaped not a man."],
['Judges','Jdg','03','030','30',"So Moab was subdued that day under the hand of Israel. And the land had rest fourscore years."],
['Judges','Jdg','03','031','31',"And after him was Shamgar the son of Anath, which slew of the Philistines six hundred men with an ox goad: and he also delivered Israel."]]

Judges04 = [
    ['Judges','Jdg','04','001','1',"And the children of Israel again did evil in the sight of the LORD, when Ehud was dead."],
['Judges','Jdg','04','002','2',"And the LORD sold them into the hand of Jabin king of Canaan, that reigned in Hazor; the captain of whose host \emph{was} Sisera, which dwelt in Harosheth of the Gentiles."],
['Judges','Jdg','04','003','3',"And the children of Israel cried unto the LORD: for he had nine hundred chariots of iron; and twenty years he mightily oppressed the children of Israel."],
['Judges','Jdg','04','004','4',"And Deborah, a prophetess, the wife of Lapidoth, she judged Israel at that time."],
['Judges','Jdg','04','005','5',"And she dwelt under the palm tree of Deborah between Ramah and Beth-el in mount Ephraim: and the children of Israel came up to her for judgment."],
['Judges','Jdg','04','006','6',"And she sent and called Barak the son of Abinoam out of Kedesh-naphtali, and said unto him, Hath not the LORD God of Israel commanded, \emph{saying}, Go and draw toward mount Tabor, and take with thee ten thousand men of the children of Naphtali and of the children of Zebulun?"],
['Judges','Jdg','04','007','7',"And I will draw unto thee to the river Kishon Sisera, the captain of Jabin's army, with his chariots and his multitude; and I will deliver him into thine hand."],
['Judges','Jdg','04','008','8',"And Barak said unto her, If thou wilt go with me, then I will go: but if thou wilt not go with me, \emph{then} I will not go."],
['Judges','Jdg','04','009','9',"And she said, I will surely go with thee: notwithstanding the journey that thou takest shall not be for thine honour; for the LORD shall sell Sisera into the hand of a woman. And Deborah arose, and went with Barak to Kedesh."],
['Judges','Jdg','04','010','10',"And Barak called Zebulun and Naphtali to Kedesh; and he went up with ten thousand men at his feet: and Deborah went up with him."],
['Judges','Jdg','04','011','11',"Now Heber the Kenite, \emph{which} \emph{was} of the children of Hobab the father in law of Moses, had severed himself from the Kenites, and pitched his tent unto the plain of Zaanaim, which \emph{is} by Kedesh."],
['Judges','Jdg','04','012','12',"And they shewed Sisera that Barak the son of Abinoam was gone up to mount Tabor."],
['Judges','Jdg','04','013','13',"And Sisera gathered together all his chariots, \emph{even} nine hundred chariots of iron, and all the people that \emph{were} with him, from Harosheth of the Gentiles unto the river of Kishon."],
['Judges','Jdg','04','014','14',"And Deborah said unto Barak, Up; for this \emph{is} the day in which the LORD hath delivered Sisera into thine hand: is not the LORD gone out before thee? So Barak went down from mount Tabor, and ten thousand men after him."],
['Judges','Jdg','04','015','15',"And the LORD discomfited Sisera, and all \emph{his} chariots, and all \emph{his} host, with the edge of the sword before Barak; so that Sisera lighted down off \emph{his} chariot, and fled away on his feet."],
['Judges','Jdg','04','016','16',"But Barak pursued after the chariots, and after the host, unto Harosheth of the Gentiles: and all the host of Sisera fell upon the edge of the sword; \emph{and} there was not a man left."],
['Judges','Jdg','04','017','17',"Howbeit Sisera fled away on his feet to the tent of Jael the wife of Heber the Kenite: for \emph{there} \emph{was} peace between Jabin the king of Hazor and the house of Heber the Kenite."],
['Judges','Jdg','04','018','18',"And Jael went out to meet Sisera, and said unto him, Turn in, my lord, turn in to me; fear not. And when he had turned in unto her into the tent, she covered him with a mantle."],
['Judges','Jdg','04','019','19',"And he said unto her, Give me, I pray thee, a little water to drink; for I am thirsty. And she opened a bottle of milk, and gave him drink, and covered him."],
['Judges','Jdg','04','020','20',"Again he said unto her, Stand in the door of the tent, and it shall be, when any man doth come and enquire of thee, and say, Is there any man here? that thou shalt say, No."],
['Judges','Jdg','04','021','21',"Then Jael Heber's wife took a nail of the tent, and took an hammer in her hand, and went softly unto him, and smote the nail into his temples, and fastened it into the ground: for he was fast asleep and weary. So he died."],
['Judges','Jdg','04','022','22',"And, behold, as Barak pursued Sisera, Jael came out to meet him, and said unto him, Come, and I will shew thee the man whom thou seekest. And when he came into her \emph{tent}, behold, Sisera lay dead, and the nail \emph{was} in his temples."],
['Judges','Jdg','04','023','23',"So God subdued on that day Jabin the king of Canaan before the children of Israel."],
['Judges','Jdg','04','024','24',"And the hand of the children of Israel prospered, and prevailed against Jabin the king of Canaan, until they had destroyed Jabin king of Canaan."]]

Judges05 = [
    ['Judges','Jdg','05','001','1',"Then sang Deborah and Barak the son of Abinoam on that day, saying,"],
['Judges','Jdg','05','002','2',"Praise ye the LORD for the avenging of Israel, when the people willingly offered themselves."],
['Judges','Jdg','05','003','3',"Hear, O ye kings; give ear, O ye princes; I, \emph{even} I, will sing unto the LORD; I will sing \emph{praise} to the LORD God of Israel."],
['Judges','Jdg','05','004','4',"LORD, when thou wentest out of Seir, when thou marchedst out of the field of Edom, the earth trembled, and the heavens dropped, the clouds also dropped water."],
['Judges','Jdg','05','005','5',"The mountains melted from before the LORD, \emph{even} that Sinai from before the LORD God of Israel."],
['Judges','Jdg','05','006','6',"In the days of Shamgar the son of Anath, in the days of Jael, the highways were unoccupied, and the travellers walked through byways."],
['Judges','Jdg','05','007','7',"\emph{The} \emph{inhabitants} \emph{of} the villages ceased, they ceased in Israel, until that I Deborah arose, that I arose a mother in Israel."],
['Judges','Jdg','05','008','8',"They chose new gods; then \emph{was} war in the gates: was there a shield or spear seen among forty thousand in Israel?"],
['Judges','Jdg','05','009','9',"My heart \emph{is} toward the governors of Israel, that offered themselves willingly among the people. Bless ye the LORD."],
['Judges','Jdg','05','010','10',"Speak, ye that ride on white asses, ye that sit in judgment, and walk by the way."],
['Judges','Jdg','05','011','11',"\emph{They} \emph{that} \emph{are} \emph{delivered} from the noise of archers in the places of drawing water, there shall they rehearse the righteous acts of the LORD, \emph{even} the righteous acts \emph{toward} \emph{the} \emph{inhabitants} of his villages in Israel: then shall the people of the LORD go down to the gates."],
['Judges','Jdg','05','012','12',"Awake, awake, Deborah: awake, awake, utter a song: arise, Barak, and lead thy captivity captive, thou son of Abinoam."],
['Judges','Jdg','05','013','13',"Then he made him that remaineth have dominion over the nobles among the people: the LORD made me have dominion over the mighty."],
['Judges','Jdg','05','014','14',"Out of Ephraim \emph{was} \emph{there} a root of them against Amalek; after thee, Benjamin, among thy people; out of Machir came down governors, and out of Zebulun they that handle the pen of the writer."],
['Judges','Jdg','05','015','15',"And the princes of Issachar \emph{were} with Deborah; even Issachar, and also Barak: he was sent on foot into the valley. For the divisions of Reuben \emph{there} \emph{were} great thoughts of heart."],
['Judges','Jdg','05','016','16',"Why abodest thou among the sheepfolds, to hear the bleatings of the flocks? For the divisions of Reuben \emph{there} \emph{were} great searchings of heart."],
['Judges','Jdg','05','017','17',"Gilead abode beyond Jordan: and why did Dan remain in ships? Asher continued on the sea shore, and abode in his breaches."],
['Judges','Jdg','05','018','18',"Zebulun and Naphtali \emph{were} a people \emph{that} jeoparded their lives unto the death in the high places of the field."],
['Judges','Jdg','05','019','19',"The kings came \emph{and} fought, then fought the kings of Canaan in Taanach by the waters of Megiddo; they took no gain of money."],
['Judges','Jdg','05','020','20',"They fought from heaven; the stars in their courses fought against Sisera."],
['Judges','Jdg','05','021','21',"The river of Kishon swept them away, that ancient river, the river Kishon. O my soul, thou hast trodden down strength."],
['Judges','Jdg','05','022','22',"Then were the horsehoofs broken by the means of the pransings, the pransings of their mighty ones."],
['Judges','Jdg','05','023','23',"Curse ye Meroz, said the angel of the LORD, curse ye bitterly the inhabitants thereof; because they came not to the help of the LORD, to the help of the LORD against the mighty."],
['Judges','Jdg','05','024','24',"Blessed above women shall Jael the wife of Heber the Kenite be, blessed shall she be above women in the tent."],
['Judges','Jdg','05','025','25',"He asked water, \emph{and} she gave \emph{him} milk; she brought forth butter in a lordly dish."],
['Judges','Jdg','05','026','26',"She put her hand to the nail, and her right hand to the workmen's hammer; and with the hammer she smote Sisera, she smote off his head, when she had pierced and stricken through his temples."],
['Judges','Jdg','05','027','27',"At her feet he bowed, he fell, he lay down: at her feet he bowed, he fell: where he bowed, there he fell down dead."],
['Judges','Jdg','05','028','28',"The mother of Sisera looked out at a window, and cried through the lattice, Why is his chariot \emph{so} long in coming? why tarry the wheels of his chariots?"],
['Judges','Jdg','05','029','29',"Her wise ladies answered her, yea, she returned answer to herself,"],
['Judges','Jdg','05','030','30',"Have they not sped? have they \emph{not} divided the prey; to every man a damsel \emph{or} two; to Sisera a prey of divers colours, a prey of divers colours of needlework, of divers colours of needlework on both sides, \emph{meet} for the necks of \emph{them} \emph{that} \emph{take} the spoil?"],
['Judges','Jdg','05','031','31',"So let all thine enemies perish, O LORD: but \emph{let} them that love him \emph{be} as the sun when he goeth forth in his might. And the land had rest forty years"]]

Judges06 = [
    ['Judges','Jdg','06','001','1',"And the children of Israel did evil in the sight of the LORD: and the LORD delivered them into the hand of Midian seven years."],
['Judges','Jdg','06','002','2',"And the hand of Midian prevailed against Israel: \emph{and} because of the Midianites the children of Israel made them the dens which \emph{are} in the mountains, and caves, and strong holds."],
['Judges','Jdg','06','003','3',"And \emph{so} it was, when Israel had sown, that the Midianites came up, and the Amalekites, and the children of the east, even they came up against them;"],
['Judges','Jdg','06','004','4',"And they encamped against them, and destroyed the increase of the earth, till thou come unto Gaza, and left no sustenance for Israel, neither sheep, nor ox, nor ass."],
['Judges','Jdg','06','005','5',"For they came up with their cattle and their tents, and they came as grasshoppers for multitude; \emph{for} both they and their camels were without number: and they entered into the land to destroy it."],
['Judges','Jdg','06','006','6',"And Israel was greatly impoverished because of the Midianites; and the children of Israel cried unto the LORD."],
['Judges','Jdg','06','007','7',"And it came to pass, when the children of Israel cried unto the LORD because of the Midianites,"],
['Judges','Jdg','06','008','8',"That the LORD sent a prophet unto the children of Israel, which said unto them, Thus saith the LORD God of Israel, I brought you up from Egypt, and brought you forth out of the house of bondage;"],
['Judges','Jdg','06','009','9',"And I delivered you out of the hand of the Egyptians, and out of the hand of all that oppressed you, and drave them out from before you, and gave you their land;"],
['Judges','Jdg','06','010','10',"And I said unto you, I \emph{am} the LORD your God; fear not the gods of the Amorites, in whose land ye dwell: but ye have not obeyed my voice."],
['Judges','Jdg','06','011','11',"And there came an angel of the LORD, and sat under an oak which \emph{was} in Ophrah, that \emph{pertained} unto Joash the Abiezrite: and his son Gideon threshed wheat by the winepress, to hide \emph{it} from the Midianites."],
['Judges','Jdg','06','012','12',"And the angel of the LORD appeared unto him, and said unto him, The LORD \emph{is} with thee, thou mighty man of valour."],
['Judges','Jdg','06','013','13',"And Gideon said unto him, Oh my Lord, if the LORD be with us, why then is all this befallen us? and where \emph{be} all his miracles which our fathers told us of, saying, Did not the LORD bring us up from Egypt? but now the LORD hath forsaken us, and delivered us into the hands of the Midianites."],
['Judges','Jdg','06','014','14',"And the LORD looked upon him, and said, Go in this thy might, and thou shalt save Israel from the hand of the Midianites: have not I sent thee?"],
['Judges','Jdg','06','015','15',"And he said unto him, Oh my Lord, wherewith shall I save Israel? behold, my family \emph{is} poor in Manasseh, and I \emph{am} the least in my father's house."],
['Judges','Jdg','06','016','16',"And the LORD said unto him, Surely I will be with thee, and thou shalt smite the Midianites as one man."],
['Judges','Jdg','06','017','17',"And he said unto him, If now I have found grace in thy sight, then shew me a sign that thou talkest with me."],
['Judges','Jdg','06','018','18',"Depart not hence, I pray thee, until I come unto thee, and bring forth my present, and set \emph{it} before thee. And he said, I will tarry until thou come again."],
['Judges','Jdg','06','019','19',"And Gideon went in, and made ready a kid, and unleavened cakes of an ephah of flour: the flesh he put in a basket, and he put the broth in a pot, and brought \emph{it} out unto him under the oak, and presented \emph{it}."],
['Judges','Jdg','06','020','20',"And the angel of God said unto him, Take the flesh and the unleavened cakes, and lay \emph{them} upon this rock, and pour out the broth. And he did so."],
['Judges','Jdg','06','021','21',"Then the angel of the LORD put forth the end of the staff that \emph{was} in his hand, and touched the flesh and the unleavened cakes; and there rose up fire out of the rock, and consumed the flesh and the unleavened cakes. Then the angel of the LORD departed out of his sight."],
['Judges','Jdg','06','022','22',"And when Gideon perceived that he \emph{was} an angel of the LORD, Gideon said, Alas, O Lord GOD! for because I have seen an angel of the LORD face to face."],
['Judges','Jdg','06','023','23',"And the LORD said unto him, Peace \emph{be} unto thee; fear not: thou shalt not die."],
['Judges','Jdg','06','024','24',"Then Gideon built an altar there unto the LORD, and called it Jehovah-shalom: unto this day it \emph{is} yet in Ophrah of the Abiezrites."],
['Judges','Jdg','06','025','25',"And it came to pass the same night, that the LORD said unto him, Take thy father's young bullock, even the second bullock of seven years old, and throw down the altar of Baal that thy father hath, and cut down the grove that \emph{is} by it:"],
['Judges','Jdg','06','026','26',"And build an altar unto the LORD thy God upon the top of this rock, in the ordered place, and take the second bullock, and offer a burnt sacrifice with the wood of the grove which thou shalt cut down."],
['Judges','Jdg','06','027','27',"Then Gideon took ten men of his servants, and did as the LORD had said unto him: and \emph{so} it was, because he feared his father's household, and the men of the city, that he could not do \emph{it} by day, that he did \emph{it} by night."],
['Judges','Jdg','06','028','28',"And when the men of the city arose early in the morning, behold, the altar of Baal was cast down, and the grove was cut down that \emph{was} by it, and the second bullock was offered upon the altar \emph{that} \emph{was} built."],
['Judges','Jdg','06','029','29',"And they said one to another, Who hath done this thing? And when they enquired and asked, they said, Gideon the son of Joash hath done this thing."],
['Judges','Jdg','06','030','30',"Then the men of the city said unto Joash, Bring out thy son, that he may die: because he hath cast down the altar of Baal, and because he hath cut down the grove that \emph{was} by it."],
['Judges','Jdg','06','031','31',"And Joash said unto all that stood against him, Will ye plead for Baal? will ye save him? he that will plead for him, let him be put to death whilst \emph{it} \emph{is} \emph{yet} morning: if he \emph{be} a god, let him plead for himself, because \emph{one} hath cast down his altar."],
['Judges','Jdg','06','032','32',"Therefore on that day he called him Jerubbaal, saying, Let Baal plead against him, because he hath thrown down his altar."],
['Judges','Jdg','06','033','33',"Then all the Midianites and the Amalekites and the children of the east were gathered together, and went over, and pitched in the valley of Jezreel."],
['Judges','Jdg','06','034','34',"But the Spirit of the LORD came upon Gideon, and he blew a trumpet; and Abiezer was gathered after him."],
['Judges','Jdg','06','035','35',"And he sent messengers throughout all Manasseh; who also was gathered after him: and he sent messengers unto Asher, and unto Zebulun, and unto Naphtali; and they came up to meet them."],
['Judges','Jdg','06','036','36',"And Gideon said unto God, If thou wilt save Israel by mine hand, as thou hast said,"],
['Judges','Jdg','06','037','37',"Behold, I will put a fleece of wool in the floor; \emph{and} if the dew be on the fleece only, and \emph{it} \emph{be} dry upon all the earth \emph{beside}, then shall I know that thou wilt save Israel by mine hand, as thou hast said."],
['Judges','Jdg','06','038','38',"And it was so: for he rose up early on the morrow, and thrust the fleece together, and wringed the dew out of the fleece, a bowl full of water."],
['Judges','Jdg','06','039','39',"And Gideon said unto God, Let not thine anger be hot against me, and I will speak but this once: let me prove, I pray thee, but this once with the fleece; let it now be dry only upon the fleece, and upon all the ground let there be dew."],
['Judges','Jdg','06','040','40',"And God did so that night: for it was dry upon the fleece only, and there was dew on all the ground."]]

Judges07 = [
    ['Judges','Jdg','07','001','1',"Then Jerubbaal, who \emph{is} Gideon, and all the people that \emph{were} with him, rose up early, and pitched beside the well of Harod: so that the host of the Midianites were on the north side of them, by the hill of Moreh, in the valley."],
['Judges','Jdg','07','002','2',"And the LORD said unto Gideon, The people that \emph{are} with thee \emph{are} too many for me to give the Midianites into their hands, lest Israel vaunt themselves against me, saying, Mine own hand hath saved me."],
['Judges','Jdg','07','003','3',"Now therefore go to, proclaim in the ears of the people, saying, Whosoever \emph{is} fearful and afraid, let him return and depart early from mount Gilead. And there returned of the people twenty and two thousand; and there remained ten thousand."],
['Judges','Jdg','07','004','4',"And the LORD said unto Gideon, The people \emph{are} yet \emph{too} many; bring them down unto the water, and I will try them for thee there: and it shall be, \emph{that} of whom I say unto thee, This shall go with thee, the same shall go with thee; and of whomsoever I say unto thee, This shall not go with thee, the same shall not go."],
['Judges','Jdg','07','005','5',"So he brought down the people unto the water: and the LORD said unto Gideon, Every one that lappeth of the water with his tongue, as a dog lappeth, him shalt thou set by himself; likewise every one that boweth down upon his knees to drink."],
['Judges','Jdg','07','006','6',"And the number of them that lapped, \emph{putting} their hand to their mouth, were three hundred men: but all the rest of the people bowed down upon their knees to drink water."],
['Judges','Jdg','07','007','7',"And the LORD said unto Gideon, By the three hundred men that lapped will I save you, and deliver the Midianites into thine hand: and let all the \emph{other} people go every man unto his place."],
['Judges','Jdg','07','008','8',"So the people took victuals in their hand, and their trumpets: and he sent all \emph{the} \emph{rest} \emph{of} Israel every man unto his tent, and retained those three hundred men: and the host of Midian was beneath him in the valley."],
['Judges','Jdg','07','009','9',"And it came to pass the same night, that the LORD said unto him, Arise, get thee down unto the host; for I have delivered it into thine hand."],
['Judges','Jdg','07','010','10',"But if thou fear to go down, go thou with Phurah thy servant down to the host:"],
['Judges','Jdg','07','011','11',"And thou shalt hear what they say; and afterward shall thine hands be strengthened to go down unto the host. Then went he down with Phurah his servant unto the outside of the armed men that \emph{were} in the host."],
['Judges','Jdg','07','012','12',"And the Midianites and the Amalekites and all the children of the east lay along in the valley like grasshoppers for multitude; and their camels \emph{were} without number, as the sand by the sea side for multitude."],
['Judges','Jdg','07','013','13',"And when Gideon was come, behold, \emph{there} \emph{was} a man that told a dream unto his fellow, and said, Behold, I dreamed a dream, and, lo, a cake of barley bread tumbled into the host of Midian, and came unto a tent, and smote it that it fell, and overturned it, that the tent lay along."],
['Judges','Jdg','07','014','14',"And his fellow answered and said, This \emph{is} nothing else save the sword of Gideon the son of Joash, a man of Israel: \emph{for} into his hand hath God delivered Midian, and all the host."],
['Judges','Jdg','07','015','15',"And it was \emph{so}, when Gideon heard the telling of the dream, and the interpretation thereof, that he worshipped, and returned into the host of Israel, and said, Arise; for the LORD hath delivered into your hand the host of Midian."],
['Judges','Jdg','07','016','16',"And he divided the three hundred men \emph{into} three companies, and he put a trumpet in every man's hand, with empty pitchers, and lamps within the pitchers."],
['Judges','Jdg','07','017','17',"And he said unto them, Look on me, and do likewise: and, behold, when I come to the outside of the camp, it shall be \emph{that}, as I do, so shall ye do."],
['Judges','Jdg','07','018','18',"When I blow with a trumpet, I and all that \emph{are} with me, then blow ye the trumpets also on every side of all the camp, and say, \emph{The} \emph{sword} of the LORD, and of Gideon."],
['Judges','Jdg','07','019','19',"So Gideon, and the hundred men that \emph{were} with him, came unto the outside of the camp in the beginning of the middle watch; and they had but newly set the watch: and they blew the trumpets, and brake the pitchers that \emph{were} in their hands."],
['Judges','Jdg','07','020','20',"And the three companies blew the trumpets, and brake the pitchers, and held the lamps in their left hands, and the trumpets in their right hands to blow \emph{withal}: and they cried, The sword of the LORD, and of Gideon."],
['Judges','Jdg','07','021','21',"And they stood every man in his place round about the camp: and all the host ran, and cried, and fled."],
['Judges','Jdg','07','022','22',"And the three hundred blew the trumpets, and the LORD set every man's sword against his fellow, even throughout all the host: and the host fled to Beth-shittah in Zererath, \emph{and} to the border of Abel-meholah, unto Tabbath."],
['Judges','Jdg','07','023','23',"And the men of Israel gathered themselves together out of Naphtali, and out of Asher, and out of all Manasseh, and pursued after the Midianites."],
['Judges','Jdg','07','024','24',"And Gideon sent messengers throughout all mount Ephraim, saying, Come down against the Midianites, and take before them the waters unto Beth-barah and Jordan. Then all the men of Ephraim gathered themselves together, and took the waters unto Beth-barah and Jordan."],
['Judges','Jdg','07','025','25',"And they took two princes of the Midianites, Oreb and Zeeb; and they slew Oreb upon the rock Oreb, and Zeeb they slew at the winepress of Zeeb, and pursued Midian, and brought the heads of Oreb and Zeeb to Gideon on the other side Jordan."]]

Judges08 = [
    ['Judges','Jdg','08','001','1',"And the men of Ephraim said unto him, Why hast thou served us thus, that thou calledst us not, when thou wentest to fight with the Midianites? And they did chide with him sharply."],
['Judges','Jdg','08','002','2',"And he said unto them, What have I done now in comparison of you? \emph{Is} not the gleaning of the grapes of Ephraim better than the vintage of Abiezer?"],
['Judges','Jdg','08','003','3',"God hath delivered into your hands the princes of Midian, Oreb and Zeeb: and what was I able to do in comparison of you? Then their anger was abated toward him, when he had said that."],
['Judges','Jdg','08','004','4',"And Gideon came to Jordan, \emph{and} passed over, he, and the three hundred men that \emph{were} with him, faint, yet pursuing \emph{them}."],
['Judges','Jdg','08','005','5',"And he said unto the men of Succoth, Give, I pray you, loaves of bread unto the people that follow me; for they \emph{be} faint, and I am pursuing after Zebah and Zalmunna, kings of Midian."],
['Judges','Jdg','08','006','6',"And the princes of Succoth said, \emph{Are} the hands of Zebah and Zalmunna now in thine hand, that we should give bread unto thine army?"],
['Judges','Jdg','08','007','7',"And Gideon said, Therefore when the LORD hath delivered Zebah and Zalmunna into mine hand, then I will tear your flesh with the thorns of the wilderness and with briers."],
['Judges','Jdg','08','008','8',"And he went up thence to Penuel, and spake unto them likewise: and the men of Penuel answered him as the men of Succoth had answered \emph{him}."],
['Judges','Jdg','08','009','9',"And he spake also unto the men of Penuel, saying, When I come again in peace, I will break down this tower."],
['Judges','Jdg','08','010','10',"Now Zebah and Zalmunna \emph{were} in Karkor, and their hosts with them, about fifteen thousand \emph{men}, all that were left of all the hosts of the children of the east: for there fell an hundred and twenty thousand men that drew sword."],
['Judges','Jdg','08','011','11',"And Gideon went up by the way of them that dwelt in tents on the east of Nobah and Jogbehah, and smote the host: for the host was secure."],
['Judges','Jdg','08','012','12',"And when Zebah and Zalmunna fled, he pursued after them, and took the two kings of Midian, Zebah and Zalmunna, and discomfited all the host."],
['Judges','Jdg','08','013','13',"And Gideon the son of Joash returned from battle before the sun \emph{was} \emph{up},"],
['Judges','Jdg','08','014','14',"And caught a young man of the men of Succoth, and enquired of him: and he described unto him the princes of Succoth, and the elders thereof, \emph{even} threescore and seventeen men."],
['Judges','Jdg','08','015','15',"And he came unto the men of Succoth, and said, Behold Zebah and Zalmunna, with whom ye did upbraid me, saying, \emph{Are} the hands of Zebah and Zalmunna now in thine hand, that we should give bread unto thy men \emph{that} \emph{are} weary?"],
['Judges','Jdg','08','016','16',"And he took the elders of the city, and thorns of the wilderness and briers, and with them he taught the men of Succoth."],
['Judges','Jdg','08','017','17',"And he beat down the tower of Penuel, and slew the men of the city."],
['Judges','Jdg','08','018','18',"Then said he unto Zebah and Zalmunna, What manner of men \emph{were} \emph{they} whom ye slew at Tabor? And they answered, As thou \emph{art}, so \emph{were} they; each one resembled the children of a king."],
['Judges','Jdg','08','019','19',"And he said, They \emph{were} my brethren, \emph{even} the sons of my mother: \emph{as} the LORD liveth, if ye had saved them alive, I would not slay you."],
['Judges','Jdg','08','020','20',"And he said unto Jether his firstborn, Up, \emph{and} slay them. But the youth drew not his sword: for he feared, because he \emph{was} yet a youth."],
['Judges','Jdg','08','021','21',"Then Zebah and Zalmunna said, Rise thou, and fall upon us: for as the man \emph{is,} \emph{so} \emph{is} his strength. And Gideon arose, and slew Zebah and Zalmunna, and took away the ornaments that \emph{were} on their camels' necks."],
['Judges','Jdg','08','022','22',"Then the men of Israel said unto Gideon, Rule thou over us, both thou, and thy son, and thy son's son also: for thou hast delivered us from the hand of Midian."],
['Judges','Jdg','08','023','23',"And Gideon said unto them, I will not rule over you, neither shall my son rule over you: the LORD shall rule over you."],
['Judges','Jdg','08','024','24',"And Gideon said unto them, I would desire a request of you, that ye would give me every man the earrings of his prey. (For they had golden earrings, because they \emph{were} Ishmaelites.)"],
['Judges','Jdg','08','025','25',"And they answered, We will willingly give \emph{them}. And they spread a garment, and did cast therein every man the earrings of his prey."],
['Judges','Jdg','08','026','26',"And the weight of the golden earrings that he requested was a thousand and seven hundred \emph{shekels} of gold; beside ornaments, and collars, and purple raiment that \emph{was} on the kings of Midian, and beside the chains that \emph{were} about their camels' necks."],
['Judges','Jdg','08','027','27',"And Gideon made an ephod thereof, and put it in his city, \emph{even} in Ophrah: and all Israel went thither a whoring after it: which thing became a snare unto Gideon, and to his house."],
['Judges','Jdg','08','028','28',"Thus was Midian subdued before the children of Israel, so that they lifted up their heads no more. And the country was in quietness forty years in the days of Gideon."],
['Judges','Jdg','08','029','29',"And Jerubbaal the son of Joash went and dwelt in his own house."],
['Judges','Jdg','08','030','30',"And Gideon had threescore and ten sons of his body begotten: for he had many wives."],
['Judges','Jdg','08','031','31',"And his concubine that \emph{was} in Shechem, she also bare him a son, whose name he called Abimelech."],
['Judges','Jdg','08','032','32',"And Gideon the son of Joash died in a good old age, and was buried in the sepulchre of Joash his father, in Ophrah of the Abiezrites."],
['Judges','Jdg','08','033','33',"And it came to pass, as soon as Gideon was dead, that the children of Israel turned again, and went a whoring after Baalim, and made Baal-berith their god."],
['Judges','Jdg','08','034','34',"And the children of Israel remembered not the LORD their God, who had delivered them out of the hands of all their enemies on every side:"],
['Judges','Jdg','08','035','35',"Neither shewed they kindness to the house of Jerubbaal, \emph{namely}, Gideon, according to all the goodness which he had shewed unto Israel."]]


Judges09 = [
    ['Judges','Jdg','09','001','1',"And Abimelech the son of Jerubbaal went to Shechem unto his mother's brethren, and communed with them, and with all the family of the house of his mother's father, saying,"],
['Judges','Jdg','09','002','2',"Speak, I pray you, in the ears of all the men of Shechem, Whether \emph{is} better for you, either that all the sons of Jerubbaal, \emph{which} \emph{are} threescore and ten persons, reign over you, or that one reign over you? remember also that I \emph{am} your bone and your flesh."],
['Judges','Jdg','09','003','3',"And his mother's brethren spake of him in the ears of all the men of Shechem all these words: and their hearts inclined to follow Abimelech; for they said, He \emph{is} our brother."],
['Judges','Jdg','09','004','4',"And they gave him threescore and ten \emph{pieces} of silver out of the house of Baal-berith, wherewith Abimelech hired vain and light persons, which followed him."],
['Judges','Jdg','09','005','5',"And he went unto his father's house at Ophrah, and slew his brethren the sons of Jerubbaal, \emph{being} threescore and ten persons, upon one stone: notwithstanding yet Jotham the youngest son of Jerubbaal was left; for he hid himself."],
['Judges','Jdg','09','006','6',"And all the men of Shechem gathered together, and all the house of Millo, and went, and made Abimelech king, by the plain of the pillar that \emph{was} in Shechem."],
['Judges','Jdg','09','007','7',"And when they told \emph{it} to Jotham, he went and stood in the top of mount Gerizim, and lifted up his voice, and cried, and said unto them, Hearken unto me, ye men of Shechem, that God may hearken unto you."],
['Judges','Jdg','09','008','8',"The trees went forth \emph{on} \emph{a} \emph{time} to anoint a king over them; and they said unto the olive tree, Reign thou over us."],
['Judges','Jdg','09','009','9',"But the olive tree said unto them, Should I leave my fatness, wherewith by me they honour God and man, and go to be promoted over the trees?"],
['Judges','Jdg','09','010','10',"And the trees said to the fig tree, Come thou, \emph{and} reign over us."],
['Judges','Jdg','09','011','11',"But the fig tree said unto them, Should I forsake my sweetness, and my good fruit, and go to be promoted over the trees?"],
['Judges','Jdg','09','012','12',"Then said the trees unto the vine, Come thou, \emph{and} reign over us."],
['Judges','Jdg','09','013','13',"And the vine said unto them, Should I leave my wine, which cheereth God and man, and go to be promoted over the trees?"],
['Judges','Jdg','09','014','14',"Then said all the trees unto the bramble, Come thou, \emph{and} reign over us."],
['Judges','Jdg','09','015','15',"And the bramble said unto the trees, If in truth ye anoint me king over you, \emph{then} come \emph{and} put your trust in my shadow: and if not, let fire come out of the bramble, and devour the cedars of Lebanon."],
['Judges','Jdg','09','016','16',"Now therefore, if ye have done truly and sincerely, in that ye have made Abimelech king, and if ye have dealt well with Jerubbaal and his house, and have done unto him according to the deserving of his hands;"],
['Judges','Jdg','09','017','17',"(For my father fought for you, and adventured his life far, and delivered you out of the hand of Midian:"],
['Judges','Jdg','09','018','18',"And ye are risen up against my father's house this day, and have slain his sons, threescore and ten persons, upon one stone, and have made Abimelech, the son of his maidservant, king over the men of Shechem, because he \emph{is} your brother;)"],
['Judges','Jdg','09','019','19',"If ye then have dealt truly and sincerely with Jerubbaal and with his house this day, \emph{then} rejoice ye in Abimelech, and let him also rejoice in you:"],
['Judges','Jdg','09','020','20',"But if not, let fire come out from Abimelech, and devour the men of Shechem, and the house of Millo; and let fire come out from the men of Shechem, and from the house of Millo, and devour Abimelech."],
['Judges','Jdg','09','021','21',"And Jotham ran away, and fled, and went to Beer, and dwelt there, for fear of Abimelech his brother."],
['Judges','Jdg','09','022','22',"When Abimelech had reigned three years over Israel,"],
['Judges','Jdg','09','023','23',"Then God sent an evil spirit between Abimelech and the men of Shechem; and the men of Shechem dealt treacherously with Abimelech:"],
['Judges','Jdg','09','024','24',"That the cruelty \emph{done} to the threescore and ten sons of Jerubbaal might come, and their blood be laid upon Abimelech their brother, which slew them; and upon the men of Shechem, which aided him in the killing of his brethren."],
['Judges','Jdg','09','025','25',"And the men of Shechem set liers in wait for him in the top of the mountains, and they robbed all that came along that way by them: and it was told Abimelech."],
['Judges','Jdg','09','026','26',"And Gaal the son of Ebed came with his brethren, and went over to Shechem: and the men of Shechem put their confidence in him."],
['Judges','Jdg','09','027','27',"And they went out into the fields, and gathered their vineyards, and trode \emph{the} \emph{grapes}, and made merry, and went into the house of their god, and did eat and drink, and cursed Abimelech."],
['Judges','Jdg','09','028','28',"And Gaal the son of Ebed said, Who \emph{is} Abimelech, and who \emph{is} Shechem, that we should serve him? \emph{is} not \emph{he} the son of Jerubbaal? and Zebul his officer? serve the men of Hamor the father of Shechem: for why should we serve him?"],
['Judges','Jdg','09','029','29',"And would to God this people were under my hand! then would I remove Abimelech. And he said to Abimelech, Increase thine army, and come out."],
['Judges','Jdg','09','030','30',"And when Zebul the ruler of the city heard the words of Gaal the son of Ebed, his anger was kindled."],
['Judges','Jdg','09','031','31',"And he sent messengers unto Abimelech privily, saying, Behold, Gaal the son of Ebed and his brethren be come to Shechem; and, behold, they fortify the city against thee."],
['Judges','Jdg','09','032','32',"Now therefore up by night, thou and the people that \emph{is} with thee, and lie in wait in the field:"],
['Judges','Jdg','09','033','33',"And it shall be, \emph{that} in the morning, as soon as the sun is up, thou shalt rise early, and set upon the city: and, behold, \emph{when} he and the people that \emph{is} with him come out against thee, then mayest thou do to them as thou shalt find occasion."],
['Judges','Jdg','09','034','34',"And Abimelech rose up, and all the people that \emph{were} with him, by night, and they laid wait against Shechem in four companies."],
['Judges','Jdg','09','035','35',"And Gaal the son of Ebed went out, and stood in the entering of the gate of the city: and Abimelech rose up, and the people that \emph{were} with him, from lying in wait."],
['Judges','Jdg','09','036','36',"And when Gaal saw the people, he said to Zebul, Behold, there come people down from the top of the mountains. And Zebul said unto him, Thou seest the shadow of the mountains as \emph{if} \emph{they} \emph{were} men."],
['Judges','Jdg','09','037','37',"And Gaal spake again and said, See there come people down by the middle of the land, and another company come along by the plain of Meonenim."],
['Judges','Jdg','09','038','38',"Then said Zebul unto him, Where \emph{is} now thy mouth, wherewith thou saidst, Who \emph{is} Abimelech, that we should serve him? \emph{is} not this the people that thou hast despised? go out, I pray now, and fight with them."],
['Judges','Jdg','09','039','39',"And Gaal went out before the men of Shechem, and fought with Abimelech."],
['Judges','Jdg','09','040','40',"And Abimelech chased him, and he fled before him, and many were overthrown \emph{and} wounded, \emph{even} unto the entering of the gate."],
['Judges','Jdg','09','041','41',"And Abimelech dwelt at Arumah: and Zebul thrust out Gaal and his brethren, that they should not dwell in Shechem."],
['Judges','Jdg','09','042','42',"And it came to pass on the morrow, that the people went out into the field; and they told Abimelech."],
['Judges','Jdg','09','043','43',"And he took the people, and divided them into three companies, and laid wait in the field, and looked, and, behold, the people \emph{were} come forth out of the city; and he rose up against them, and smote them."],
['Judges','Jdg','09','044','44',"And Abimelech, and the company that \emph{was} with him, rushed forward, and stood in the entering of the gate of the city: and the two \emph{other} companies ran upon all \emph{the} \emph{people} that \emph{were} in the fields, and slew them."],
['Judges','Jdg','09','045','45',"And Abimelech fought against the city all that day; and he took the city, and slew the people that \emph{was} therein, and beat down the city, and sowed it with salt."],
['Judges','Jdg','09','046','46',"And when all the men of the tower of Shechem heard \emph{that}, they entered into an hold of the house of the god Berith."],
['Judges','Jdg','09','047','47',"And it was told Abimelech, that all the men of the tower of Shechem were gathered together."],
['Judges','Jdg','09','048','48',"And Abimelech gat him up to mount Zalmon, he and all the people that \emph{were} with him; and Abimelech took an axe in his hand, and cut down a bough from the trees, and took it, and laid \emph{it} on his shoulder, and said unto the people that \emph{were} with him, What ye have seen me do, make haste, \emph{and} do as I \emph{have} \emph{done}."],
['Judges','Jdg','09','049','49',"And all the people likewise cut down every man his bough, and followed Abimelech, and put \emph{them} to the hold, and set the hold on fire upon them; so that all the men of the tower of Shechem died also, about a thousand men and women."],
['Judges','Jdg','09','050','50',"Then went Abimelech to Thebez, and encamped against Thebez, and took it."],
['Judges','Jdg','09','051','51',"But there was a strong tower within the city, and thither fled all the men and women, and all they of the city, and shut \emph{it} to them, and gat them up to the top of the tower."],
['Judges','Jdg','09','052','52',"And Abimelech came unto the tower, and fought against it, and went hard unto the door of the tower to burn it with fire."],
['Judges','Jdg','09','053','53',"And a certain woman cast a piece of a millstone upon Abimelech's head, and all to brake his skull."],
['Judges','Jdg','09','054','54',"Then he called hastily unto the young man his armourbearer, and said unto him, Draw thy sword, and slay me, that men say not of me, A woman slew him. And his young man thrust him through, and he died."],
['Judges','Jdg','09','055','55',"And when the men of Israel saw that Abimelech was dead, they departed every man unto his place."],
['Judges','Jdg','09','056','56',"Thus God rendered the wickedness of Abimelech, which he did unto his father, in slaying his seventy brethren:"],
['Judges','Jdg','09','057','57',"And all the evil of the men of Shechem did God render upon their heads: and upon them came the curse of Jotham the son of Jerubbaal."]]

Judges10 = [
    ['Judges','Jdg','10','001','1',"And after Abimelech there arose to defend Israel Tola the son of Puah, the son of Dodo, a man of Issachar; and he dwelt in Shamir in mount Ephraim."],
['Judges','Jdg','10','002','2',"And he judged Israel twenty and three years, and died, and was buried in Shamir."],
['Judges','Jdg','10','003','3',"And after him arose Jair, a Gileadite, and judged Israel twenty and two years."],
['Judges','Jdg','10','004','4',"And he had thirty sons that rode on thirty ass colts, and they had thirty cities, which are called Havoth-jair unto this day, which \emph{are} in the land of Gilead."],
['Judges','Jdg','10','005','5',"And Jair died, and was buried in Camon."],
['Judges','Jdg','10','006','6',"And the children of Israel did evil again in the sight of the LORD, and served Baalim, and Ashtaroth, and the gods of Syria, and the gods of Zidon, and the gods of Moab, and the gods of the children of Ammon, and the gods of the Philistines, and forsook the LORD, and served not him."],
['Judges','Jdg','10','007','7',"And the anger of the LORD was hot against Israel, and he sold them into the hands of the Philistines, and into the hands of the children of Ammon."],
['Judges','Jdg','10','008','8',"And that year they vexed and oppressed the children of Israel: eighteen years, all the children of Israel that \emph{were} on the other side Jordan in the land of the Amorites, which \emph{is} in Gilead."],
['Judges','Jdg','10','009','9',"Moreover the children of Ammon passed over Jordan to fight also against Judah, and against Benjamin, and against the house of Ephraim; so that Israel was sore distressed."],
['Judges','Jdg','10','010','10',"And the children of Israel cried unto the LORD, saying, We have sinned against thee, both because we have forsaken our God, and also served Baalim."],
['Judges','Jdg','10','011','11',"And the LORD said unto the children of Israel, \emph{Did} not \emph{I} \emph{deliver} \emph{you} from the Egyptians, and from the Amorites, from the children of Ammon, and from the Philistines?"],
['Judges','Jdg','10','012','12',"The Zidonians also, and the Amalekites, and the Maonites, did oppress you; and ye cried to me, and I delivered you out of their hand."],
['Judges','Jdg','10','013','13',"Yet ye have forsaken me, and served other gods: wherefore I will deliver you no more."],
['Judges','Jdg','10','014','14',"Go and cry unto the gods which ye have chosen; let them deliver you in the time of your tribulation."],
['Judges','Jdg','10','015','15',"And the children of Israel said unto the LORD, We have sinned: do thou unto us whatsoever seemeth good unto thee; deliver us only, we pray thee, this day."],
['Judges','Jdg','10','016','16',"And they put away the strange gods from among them, and served the LORD: and his soul was grieved for the misery of Israel."],
['Judges','Jdg','10','017','17',"Then the children of Ammon were gathered together, and encamped in Gilead. And the children of Israel assembled themselves together, and encamped in Mizpeh."],
['Judges','Jdg','10','018','18',"And the people \emph{and} princes of Gilead said one to another, What man \emph{is} \emph{he} that will begin to fight against the children of Ammon? he shall be head over all the inhabitants of Gilead."]]

Judges11 = [
    ['Judges','Jdg','11','001','1',"Now Jephthah the Gileadite was a mighty man of valour, and he \emph{was} the son of an harlot: and Gilead begat Jephthah."],
['Judges','Jdg','11','002','2',"And Gilead's wife bare him sons; and his wife's sons grew up, and they thrust out Jephthah, and said unto him, Thou shalt not inherit in our father's house; for thou \emph{art} the son of a strange woman."],
['Judges','Jdg','11','003','3',"Then Jephthah fled from his brethren, and dwelt in the land of Tob: and there were gathered vain men to Jephthah, and went out with him."],
['Judges','Jdg','11','004','4',"And it came to pass in process of time, that the children of Ammon made war against Israel."],
['Judges','Jdg','11','005','5',"And it was so, that when the children of Ammon made war against Israel, the elders of Gilead went to fetch Jephthah out of the land of Tob:"],
['Judges','Jdg','11','006','6',"And they said unto Jephthah, Come, and be our captain, that we may fight with the children of Ammon."],
['Judges','Jdg','11','007','7',"And Jephthah said unto the elders of Gilead, Did not ye hate me, and expel me out of my father's house? and why are ye come unto me now when ye are in distress?"],
['Judges','Jdg','11','008','8',"And the elders of Gilead said unto Jephthah, Therefore we turn again to thee now, that thou mayest go with us, and fight against the children of Ammon, and be our head over all the inhabitants of Gilead."],
['Judges','Jdg','11','009','9',"And Jephthah said unto the elders of Gilead, If ye bring me home again to fight against the children of Ammon, and the LORD deliver them before me, shall I be your head?"],
['Judges','Jdg','11','010','10',"And the elders of Gilead said unto Jephthah, The LORD be witness between us, if we do not so according to thy words."],
['Judges','Jdg','11','011','11',"Then Jephthah went with the elders of Gilead, and the people made him head and captain over them: and Jephthah uttered all his words before the LORD in Mizpeh."],
['Judges','Jdg','11','012','12',"And Jephthah sent messengers unto the king of the children of Ammon, saying, What hast thou to do with me, that thou art come against me to fight in my land?"],
['Judges','Jdg','11','013','13',"And the king of the children of Ammon answered unto the messengers of Jephthah, Because Israel took away my land, when they came up out of Egypt, from Arnon even unto Jabbok, and unto Jordan: now therefore restore those \emph{lands} again peaceably."],
['Judges','Jdg','11','014','14',"And Jephthah sent messengers again unto the king of the children of Ammon:"],
['Judges','Jdg','11','015','15',"And said unto him, Thus saith Jephthah, Israel took not away the land of Moab, nor the land of the children of Ammon:"],
['Judges','Jdg','11','016','16',"But when Israel came up from Egypt, and walked through the wilderness unto the Red sea, and came to Kadesh;"],
['Judges','Jdg','11','017','17',"Then Israel sent messengers unto the king of Edom, saying, Let me, I pray thee, pass through thy land: but the king of Edom would not hearken \emph{thereto}. And in like manner they sent unto the king of Moab: but he would not \emph{consent}: and Israel abode in Kadesh."],
['Judges','Jdg','11','018','18',"Then they went along through the wilderness, and compassed the land of Edom, and the land of Moab, and came by the east side of the land of Moab, and pitched on the other side of Arnon, but came not within the border of Moab: for Arnon \emph{was} the border of Moab."],
['Judges','Jdg','11','019','19',"And Israel sent messengers unto Sihon king of the Amorites, the king of Heshbon; and Israel said unto him, Let us pass, we pray thee, through thy land into my place."],
['Judges','Jdg','11','020','20',"But Sihon trusted not Israel to pass through his coast: but Sihon gathered all his people together, and pitched in Jahaz, and fought against Israel."],
['Judges','Jdg','11','021','21',"And the LORD God of Israel delivered Sihon and all his people into the hand of Israel, and they smote them: so Israel possessed all the land of the Amorites, the inhabitants of that country."],
['Judges','Jdg','11','022','22',"And they possessed all the coasts of the Amorites, from Arnon even unto Jabbok, and from the wilderness even unto Jordan."],
['Judges','Jdg','11','023','23',"So now the LORD God of Israel hath dispossessed the Amorites from before his people Israel, and shouldest thou possess it?"],
['Judges','Jdg','11','024','24',"Wilt not thou possess that which Chemosh thy god giveth thee to possess? So whomsoever the LORD our God shall drive out from before us, them will we possess."],
['Judges','Jdg','11','025','25',"And now \emph{art} thou any thing better than Balak the son of Zippor, king of Moab? did he ever strive against Israel, or did he ever fight against them,"],
['Judges','Jdg','11','026','26',"While Israel dwelt in Heshbon and her towns, and in Aroer and her towns, and in all the cities that \emph{be} along by the coasts of Arnon, three hundred years? why therefore did ye not recover \emph{them} within that time?"],
['Judges','Jdg','11','027','27',"Wherefore I have not sinned against thee, but thou doest me wrong to war against me: the LORD the Judge be judge this day between the children of Israel and the children of Ammon."],
['Judges','Jdg','11','028','28',"Howbeit the king of the children of Ammon hearkened not unto the words of Jephthah which he sent him."],
['Judges','Jdg','11','029','29',"Then the Spirit of the LORD came upon Jephthah, and he passed over Gilead, and Manasseh, and passed over Mizpeh of Gilead, and from Mizpeh of Gilead he passed over \emph{unto} the children of Ammon."],
['Judges','Jdg','11','030','30',"And Jephthah vowed a vow unto the LORD, and said, If thou shalt without fail deliver the children of Ammon into mine hands,"],
['Judges','Jdg','11','031','31',"Then it shall be, that whatsoever cometh forth of the doors of my house to meet me, when I return in peace from the children of Ammon, shall surely be the LORD'S, and I will offer it up for a burnt offering."],
['Judges','Jdg','11','032','32',"So Jephthah passed over unto the children of Ammon to fight against them; and the LORD delivered them into his hands."],
['Judges','Jdg','11','033','33',"And he smote them from Aroer, even till thou come to Minnith, \emph{even} twenty cities, and unto the plain of the vineyards, with a very great slaughter. Thus the children of Ammon were subdued before the children of Israel."],
['Judges','Jdg','11','034','34',"And Jephthah came to Mizpeh unto his house, and, behold, his daughter came out to meet him with timbrels and with dances: and she \emph{was} \emph{his} only child; beside her he had neither son nor daughter."],
['Judges','Jdg','11','035','35',"And it came to pass, when he saw her, that he rent his clothes, and said, Alas, my daughter! thou hast brought me very low, and thou art one of them that trouble me: for I have opened my mouth unto the LORD, and I cannot go back."],
['Judges','Jdg','11','036','36',"And she said unto him, My father, \emph{if} thou hast opened thy mouth unto the LORD, do to me according to that which hath proceeded out of thy mouth; forasmuch as the LORD hath taken vengeance for thee of thine enemies, \emph{even} of the children of Ammon."],
['Judges','Jdg','11','037','37',"And she said unto her father, Let this thing be done for me: let me alone two months, that I may go up and down upon the mountains, and bewail my virginity, I and my fellows."],
['Judges','Jdg','11','038','38',"And he said, Go. And he sent her away \emph{for} two months: and she went with her companions, and bewailed her virginity upon the mountains."],
['Judges','Jdg','11','039','39',"And it came to pass at the end of two months, that she returned unto her father, who did with her \emph{according} to his vow which he had vowed: and she knew no man. And it was a custom in Israel,"],
['Judges','Jdg','11','040','40',"\emph{That} the daughters of Israel went yearly to lament the daughter of Jephthah the Gileadite four days in a year."]
]

Judges12 = [
    ['Judges','Jdg','12','001','1',"And the men of Ephraim gathered themselves together, and went northward, and said unto Jephthah, Wherefore passedst thou over to fight against the children of Ammon, and didst not call us to go with thee? we will burn thine house upon thee with fire."],
    ['Judges','Jdg','12','002','2',"And Jephthah said unto them, I and my people were at great strife with the children of Ammon; and when I called you, ye delivered me not out of their hands."],
    ['Judges','Jdg','12','003','3',"And when I saw that ye delivered \emph{me} not, I put my life in my hands, and passed over against the children of Ammon, and the LORD delivered them into my hand: wherefore then are ye come up unto me this day, to fight against me?"],
    ['Judges','Jdg','12','004','4',"Then Jephthah gathered together all the men of Gilead, and fought with Ephraim: and the men of Gilead smote Ephraim, because they said, Ye Gileadites \emph{are} fugitives of Ephraim among the Ephraimites, \emph{and} among the Manassites."],
    ['Judges','Jdg','12','005','5',"And the Gileadites took the passages of Jordan before the Ephraimites: and it was \emph{so}, that when those Ephraimites which were escaped said, Let me go over; that the men of Gilead said unto him, \emph{Art} thou an Ephraimite? If he said, Nay;"],
    ['Judges','Jdg','12','006','6',"Then said they unto him, Say now Shibboleth: and he said Sibboleth: for he could not frame to pronounce \emph{it} right. Then they took him, and slew him at the passages of Jordan: and there fell at that time of the Ephraimites forty and two thousand."],
    ['Judges','Jdg','12','007','7',"And Jephthah judged Israel six years. Then died Jephthah the Gileadite, and was buried in \emph{one} \emph{of} the cities of Gilead."],
    ['Judges','Jdg','12','008','8',"And after him Ibzan of Beth-lehem judged Israel."],
    ['Judges','Jdg','12','009','9',"And he had thirty sons, and thirty daughters, \emph{whom} he sent abroad, and took in thirty daughters from abroad for his sons. And he judged Israel seven years."],
    ['Judges','Jdg','12','010','10',"Then died Ibzan, and was buried at Beth-lehem."],
    ['Judges','Jdg','12','011','11',"And after him Elon, a Zebulonite, judged Israel; and he judged Israel ten years."],
    ['Judges','Jdg','12','012','12',"And Elon the Zebulonite died, and was buried in Aijalon in the country of Zebulun."],
    ['Judges','Jdg','12','013','13',"And after him Abdon the son of Hillel, a Pirathonite, judged Israel."],
    ['Judges','Jdg','12','014','14',"And he had forty sons and thirty nephews, that rode on threescore and ten ass colts: and he judged Israel eight years."],
    ['Judges','Jdg','12','015','15',"And Abdon the son of Hillel the Pirathonite died, and was buried in Pirathon in the land of Ephraim, in the mount of the Amalekites."]
]

Judges13 = [
    ['Judges','Jdg','13','001','1',"And the children of Israel did evil again in the sight of the LORD; and the LORD delivered them into the hand of the Philistines forty years."],
    ['Judges','Jdg','13','002','2',"And there was a certain man of Zorah, of the family of the Danites, whose name \emph{was} Manoah; and his wife \emph{was} barren, and bare not."],
    ['Judges','Jdg','13','003','3',"And the angel of the LORD appeared unto the woman, and said unto her, Behold now, thou \emph{art} barren, and bearest not: but thou shalt conceive, and bear a son."],
    ['Judges','Jdg','13','004','4',"Now therefore beware, I pray thee, and drink not wine nor strong drink, and eat not any unclean \emph{thing}:"],
    ['Judges','Jdg','13','005','5',"For, lo, thou shalt conceive, and bear a son; and no razor shall come on his head: for the child shall be a Nazarite unto God from the womb: and he shall begin to deliver Israel out of the hand of the Philistines."],
    ['Judges','Jdg','13','006','6',"Then the woman came and told her husband, saying, A man of God came unto me, and his countenance \emph{was} like the countenance of an angel of God, very terrible: but I asked him not whence he \emph{was}, neither told he me his name:"],
    ['Judges','Jdg','13','007','7',"But he said unto me, Behold, thou shalt conceive, and bear a son; and now drink no wine nor strong drink, neither eat any unclean \emph{thing}: for the child shall be a Nazarite to God from the womb to the day of his death."],
    ['Judges','Jdg','13','008','8',"Then Manoah intreated the LORD, and said, O my Lord, let the man of God which thou didst send come again unto us, and teach us what we shall do unto the child that shall be born."],
    ['Judges','Jdg','13','009','9',"And God hearkened to the voice of Manoah; and the angel of God came again unto the woman as she sat in the field: but Manoah her husband \emph{was} not with her."],
    ['Judges','Jdg','13','010','10',"And the woman made haste, and ran, and shewed her husband, and said unto him, Behold, the man hath appeared unto me, that came unto me the \emph{other} day."],
    ['Judges','Jdg','13','011','11',"And Manoah arose, and went after his wife, and came to the man, and said unto him, \emph{Art} thou the man that spakest unto the woman? And he said, I \emph{am}."],
    ['Judges','Jdg','13','012','12',"And Manoah said, Now let thy words come to pass. How shall we order the child, and \emph{how} shall we do unto him?"],
    ['Judges','Jdg','13','013','13',"And the angel of the LORD said unto Manoah, Of all that I said unto the woman let her beware."],
    ['Judges','Jdg','13','014','14',"She may not eat of any \emph{thing} that cometh of the vine, neither let her drink wine or strong drink, nor eat any unclean \emph{thing}: all that I commanded her let her observe."],
    ['Judges','Jdg','13','015','15',"And Manoah said unto the angel of the LORD, I pray thee, let us detain thee, until we shall have made ready a kid for thee."],
    ['Judges','Jdg','13','016','16',"And the angel of the LORD said unto Manoah, Though thou detain me, I will not eat of thy bread: and if thou wilt offer a burnt offering, thou must offer it unto the LORD. For Manoah knew not that he \emph{was} an angel of the LORD."],
    ['Judges','Jdg','13','017','17',"And Manoah said unto the angel of the LORD, What \emph{is} thy name, that when thy sayings come to pass we may do thee honour?"],
    ['Judges','Jdg','13','018','18',"And the angel of the LORD said unto him, Why askest thou thus after my name, seeing it \emph{is} secret?"],
    ['Judges','Jdg','13','019','19',"So Manoah took a kid with a meat offering, and offered \emph{it} upon a rock unto the LORD: and \emph{the} \emph{angel} did wondrously; and Manoah and his wife looked on."],
    ['Judges','Jdg','13','020','20',"For it came to pass, when the flame went up toward heaven from off the altar, that the angel of the LORD ascended in the flame of the altar. And Manoah and his wife looked on \emph{it}, and fell on their faces to the ground."],
    ['Judges','Jdg','13','021','21',"But the angel of the LORD did no more appear to Manoah and to his wife. Then Manoah knew that he \emph{was} an angel of the LORD."],
    ['Judges','Jdg','13','022','22',"And Manoah said unto his wife, We shall surely die, because we have seen God."],
    ['Judges','Jdg','13','023','23',"But his wife said unto him, If the LORD were pleased to kill us, he would not have received a burnt offering and a meat offering at our hands, neither would he have shewed us all these \emph{things}, nor would as at this time have told us \emph{such} \emph{things} as these."],
    ['Judges','Jdg','13','024','24',"And the woman bare a son, and called his name Samson: and the child grew, and the LORD blessed him."],
    ['Judges','Jdg','13','025','25',"And the Spirit of the LORD began to move him at times in the camp of Dan between Zorah and Eshtaol."]
]

Judges14 = [
    ['Judges','Jdg','14','001','1',"And Samson went down to Timnath, and saw a woman in Timnath of the daughters of the Philistines."],
['Judges','Jdg','14','002','2',"And he came up, and told his father and his mother, and said, I have seen a woman in Timnath of the daughters of the Philistines: now therefore get her for me to wife."],
['Judges','Jdg','14','003','3',"Then his father and his mother said unto him, \emph{Is} \emph{there} never a woman among the daughters of thy brethren, or among all my people, that thou goest to take a wife of the uncircumcised Philistines? And Samson said unto his father, Get her for me; for she pleaseth me well."],
['Judges','Jdg','14','004','4',"But his father and his mother knew not that it \emph{was} of the LORD, that he sought an occasion against the Philistines: for at that time the Philistines had dominion over Israel."],
['Judges','Jdg','14','005','5',"Then went Samson down, and his father and his mother, to Timnath, and came to the vineyards of Timnath: and, behold, a young lion roared against him."],
['Judges','Jdg','14','006','6',"And the Spirit of the LORD came mightily upon him, and he rent him as he would have rent a kid, and \emph{he} \emph{had} nothing in his hand: but he told not his father or his mother what he had done."],
['Judges','Jdg','14','007','7',"And he went down, and talked with the woman; and she pleased Samson well."],
['Judges','Jdg','14','008','8',"And after a time he returned to take her, and he turned aside to see the carcase of the lion: and, behold, \emph{there} \emph{was} a swarm of bees and honey in the carcase of the lion."],
['Judges','Jdg','14','009','9',"And he took thereof in his hands, and went on eating, and came to his father and mother, and he gave them, and they did eat: but he told not them that he had taken the honey out of the carcase of the lion."],
['Judges','Jdg','14','010','10',"So his father went down unto the woman: and Samson made there a feast; for so used the young men to do."],
['Judges','Jdg','14','011','11',"And it came to pass, when they saw him, that they brought thirty companions to be with him."],
['Judges','Jdg','14','012','12',"And Samson said unto them, I will now put forth a riddle unto you: if ye can certainly declare it me within the seven days of the feast, and find \emph{it} out, then I will give you thirty sheets and thirty change of garments:"],
['Judges','Jdg','14','013','13',"But if ye cannot declare \emph{it} me, then shall ye give me thirty sheets and thirty change of garments. And they said unto him, Put forth thy riddle, that we may hear it."],
['Judges','Jdg','14','014','14',"And he said unto them, Out of the eater came forth meat, and out of the strong came forth sweetness. And they could not in three days expound the riddle."],
['Judges','Jdg','14','015','15',"And it came to pass on the seventh day, that they said unto Samson's wife, Entice thy husband, that he may declare unto us the riddle, lest we burn thee and thy father's house with fire: have ye called us to take that we have? \emph{is} \emph{it} not \emph{so}?"],
['Judges','Jdg','14','016','16',"And Samson's wife wept before him, and said, Thou dost but hate me, and lovest me not: thou hast put forth a riddle unto the children of my people, and hast not told \emph{it} me. And he said unto her, Behold, I have not told \emph{it} my father nor my mother, and shall I tell \emph{it} thee?"],
['Judges','Jdg','14','017','17',"And she wept before him the seven days, while their feast lasted: and it came to pass on the seventh day, that he told her, because she lay sore upon him: and she told the riddle to the children of her people."],
['Judges','Jdg','14','018','18',"And the men of the city said unto him on the seventh day before the sun went down, What \emph{is} sweeter than honey? and what \emph{is} stronger than a lion? And he said unto them, If ye had not plowed with my heifer, ye had not found out my riddle."],
['Judges','Jdg','14','019','19',"And the Spirit of the LORD came upon him, and he went down to Ashkelon, and slew thirty men of them, and took their spoil, and gave change of garments unto them which expounded the riddle. And his anger was kindled, and he went up to his father's house."],
['Judges','Jdg','14','020','20',"But Samson's wife was \emph{given} to his companion, whom he had used as his friend."]
]


Judges15 = [
    ['Judges','Jdg','15','001','1',"But it came to pass within a while after, in the time of wheat harvest, that Samson visited his wife with a kid; and he said, I will go in to my wife into the chamber. But her father would not suffer him to go in."],
['Judges','Jdg','15','002','2',"And her father said, I verily thought that thou hadst utterly hated her; therefore I gave her to thy companion: \emph{is} not her younger sister fairer than she? take her, I pray thee, instead of her."],
['Judges','Jdg','15','003','3',"And Samson said concerning them, Now shall I be more blameless than the Philistines, though I do them a displeasure."],
['Judges','Jdg','15','004','4',"And Samson went and caught three hundred foxes, and took firebrands, and turned tail to tail, and put a firebrand in the midst between two tails."],
['Judges','Jdg','15','005','5',"And when he had set the brands on fire, he let \emph{them} go into the standing corn of the Philistines, and burnt up both the shocks, and also the standing corn, with the vineyards \emph{and} olives."],
['Judges','Jdg','15','006','6',"Then the Philistines said, Who hath done this? And they answered, Samson, the son in law of the Timnite, because he had taken his wife, and given her to his companion. And the Philistines came up, and burnt her and her father with fire."],
['Judges','Jdg','15','007','7',"And Samson said unto them, Though ye have done this, yet will I be avenged of you, and after that I will cease."],
['Judges','Jdg','15','008','8',"And he smote them hip and thigh with a great slaughter: and he went down and dwelt in the top of the rock Etam."],
['Judges','Jdg','15','009','9',"Then the Philistines went up, and pitched in Judah, and spread themselves in Lehi."],
['Judges','Jdg','15','010','10',"And the men of Judah said, Why are ye come up against us? And they answered, To bind Samson are we come up, to do to him as he hath done to us."],
['Judges','Jdg','15','011','11',"Then three thousand men of Judah went to the top of the rock Etam, and said to Samson, Knowest thou not that the Philistines \emph{are} rulers over us? what \emph{is} this \emph{that} thou hast done unto us? And he said unto them, As they did unto me, so have I done unto them."],
['Judges','Jdg','15','012','12',"And they said unto him, We are come down to bind thee, that we may deliver thee into the hand of the Philistines. And Samson said unto them, Swear unto me, that ye will not fall upon me yourselves."],
['Judges','Jdg','15','013','13',"And they spake unto him, saying, No; but we will bind thee fast, and deliver thee into their hand: but surely we will not kill thee. And they bound him with two new cords, and brought him up from the rock."],
['Judges','Jdg','15','014','14',"\emph{And} when he came unto Lehi, the Philistines shouted against him: and the Spirit of the LORD came mightily upon him, and the cords that \emph{were} upon his arms became as flax that was burnt with fire, and his bands loosed from off his hands."],
['Judges','Jdg','15','015','15',"And he found a new jawbone of an ass, and put forth his hand, and took it, and slew a thousand men therewith."],
['Judges','Jdg','15','016','16',"And Samson said, With the jawbone of an ass, heaps upon heaps, with the jaw of an ass have I slain a thousand men."],
['Judges','Jdg','15','017','17',"And it came to pass, when he had made an end of speaking, that he cast away the jawbone out of his hand, and called that place Ramath-lehi."],
['Judges','Jdg','15','018','18',"And he was sore athirst, and called on the LORD, and said, Thou hast given this great deliverance into the hand of thy servant: and now shall I die for thirst, and fall into the hand of the uncircumcised?"],
['Judges','Jdg','15','019','19',"But God clave an hollow place that \emph{was} in the jaw, and there came water thereout; and when he had drunk, his spirit came again, and he revived: wherefore he called the name thereof En-hakkore, which \emph{is} in Lehi unto this day."],
['Judges','Jdg','15','020','20',"And he judged Israel in the days of the Philistines twenty years."]]

Judges16 = [
    ['Judges','Jdg','16','001','1',"Then went Samson to Gaza, and saw there an harlot, and went in unto her."],
['Judges','Jdg','16','002','2',"\emph{And} \emph{it} \emph{was} \emph{told} the Gazites, saying, Samson is come hither. And they compassed \emph{him} in, and laid wait for him all night in the gate of the city, and were quiet all the night, saying, In the morning, when it is day, we shall kill him."],
['Judges','Jdg','16','003','3',"And Samson lay till midnight, and arose at midnight, and took the doors of the gate of the city, and the two posts, and went away with them, bar and all, and put \emph{them} upon his shoulders, and carried them up to the top of an hill that \emph{is} before Hebron."],
['Judges','Jdg','16','004','4',"And it came to pass afterward, that he loved a woman in the valley of Sorek, whose name \emph{was} Delilah."],
['Judges','Jdg','16','005','5',"And the lords of the Philistines came up unto her, and said unto her, Entice him, and see wherein his great strength \emph{lieth}, and by what \emph{means} we may prevail against him, that we may bind him to afflict him: and we will give thee every one of us eleven hundred \emph{pieces} of silver."],
['Judges','Jdg','16','006','6',"And Delilah said to Samson, Tell me, I pray thee, wherein thy great strength \emph{lieth}, and wherewith thou mightest be bound to afflict thee."],
['Judges','Jdg','16','007','7',"And Samson said unto her, If they bind me with seven green withs that were never dried, then shall I be weak, and be as another man."],
['Judges','Jdg','16','008','8',"Then the lords of the Philistines brought up to her seven green withs which had not been dried, and she bound him with them."],
['Judges','Jdg','16','009','9',"Now \emph{there} \emph{were} men lying in wait, abiding with her in the chamber. And she said unto him, The Philistines \emph{be} upon thee, Samson. And he brake the withs, as a thread of tow is broken when it toucheth the fire. So his strength was not known."],
['Judges','Jdg','16','010','10',"And Delilah said unto Samson, Behold, thou hast mocked me, and told me lies: now tell me, I pray thee, wherewith thou mightest be bound."],
['Judges','Jdg','16','011','11',"And he said unto her, If they bind me fast with new ropes that never were occupied, then shall I be weak, and be as another man."],
['Judges','Jdg','16','012','12',"Delilah therefore took new ropes, and bound him therewith, and said unto him, The Philistines \emph{be} upon thee, Samson. And \emph{there} \emph{were} liers in wait abiding in the chamber. And he brake them from off his arms like a thread."],
['Judges','Jdg','16','013','13',"And Delilah said unto Samson, Hitherto thou hast mocked me, and told me lies: tell me wherewith thou mightest be bound. And he said unto her, If thou weavest the seven locks of my head with the web."],
['Judges','Jdg','16','014','14',"And she fastened \emph{it} with the pin, and said unto him, The Philistines \emph{be} upon thee, Samson. And he awaked out of his sleep, and went away with the pin of the beam, and with the web."],
['Judges','Jdg','16','015','15',"And she said unto him, How canst thou say, I love thee, when thine heart \emph{is} not with me? thou hast mocked me these three times, and hast not told me wherein thy great strength \emph{lieth}."],
['Judges','Jdg','16','016','16',"And it came to pass, when she pressed him daily with her words, and urged him, \emph{so} that his soul was vexed unto death;"],
['Judges','Jdg','16','017','17',"That he told her all his heart, and said unto her, There hath not come a razor upon mine head; for I \emph{have} \emph{been} a Nazarite unto God from my mother's womb: if I be shaven, then my strength will go from me, and I shall become weak, and be like any \emph{other} man."],
['Judges','Jdg','16','018','18',"And when Delilah saw that he had told her all his heart, she sent and called for the lords of the Philistines, saying, Come up this once, for he hath shewed me all his heart. Then the lords of the Philistines came up unto her, and brought money in their hand."],
['Judges','Jdg','16','019','19',"And she made him sleep upon her knees; and she called for a man, and she caused him to shave off the seven locks of his head; and she began to afflict him, and his strength went from him."],
['Judges','Jdg','16','020','20',"And she said, The Philistines \emph{be} upon thee, Samson. And he awoke out of his sleep, and said, I will go out as at other times before, and shake myself. And he wist not that the LORD was departed from him."],
['Judges','Jdg','16','021','21',"But the Philistines took him, and put out his eyes, and brought him down to Gaza, and bound him with fetters of brass; and he did grind in the prison house."],
['Judges','Jdg','16','022','22',"Howbeit the hair of his head began to grow again after he was shaven."],
['Judges','Jdg','16','023','23',"Then the lords of the Philistines gathered them together for to offer a great sacrifice unto Dagon their god, and to rejoice: for they said, Our god hath delivered Samson our enemy into our hand."],
['Judges','Jdg','16','024','24',"And when the people saw him, they praised their god: for they said, Our god hath delivered into our hands our enemy, and the destroyer of our country, which slew many of us."],
['Judges','Jdg','16','025','25',"And it came to pass, when their hearts were merry, that they said, Call for Samson, that he may make us sport. And they called for Samson out of the prison house; and he made them sport: and they set him between the pillars."],
['Judges','Jdg','16','026','26',"And Samson said unto the lad that held him by the hand, Suffer me that I may feel the pillars whereupon the house standeth, that I may lean upon them."],
['Judges','Jdg','16','027','27',"Now the house was full of men and women; and all the lords of the Philistines \emph{were} there; and \emph{there} \emph{were} upon the roof about three thousand men and women, that beheld while Samson made sport."],
['Judges','Jdg','16','028','28',"And Samson called unto the LORD, and said, O Lord GOD, remember me, I pray thee, and strengthen me, I pray thee, only this once, O God, that I may be at once avenged of the Philistines for my two eyes."],
['Judges','Jdg','16','029','29',"And Samson took hold of the two middle pillars upon which the house stood, and on which it was borne up, of the one with his right hand, and of the other with his left."],
['Judges','Jdg','16','030','30',"And Samson said, Let me die with the Philistines. And he bowed himself with \emph{all} \emph{his} might; and the house fell upon the lords, and upon all the people that \emph{were} therein. So the dead which he slew at his death were more than \emph{they} which he slew in his life."],
['Judges','Jdg','16','031','31',"Then his brethren and all the house of his father came down, and took him, and brought \emph{him} up, and buried him between Zorah and Eshtaol in the buryingplace of Manoah his father. And he judged Israel twenty years."]]

Judges17 = [
    ['Judges','Jdg','17','001','1',"And there was a man of mount Ephraim, whose name \emph{was} Micah."],
['Judges','Jdg','17','002','2',"And he said unto his mother, The eleven hundred \emph{shekels} of silver that were taken from thee, about which thou cursedst, and spakest of also in mine ears, behold, the silver \emph{is} with me; I took it. And his mother said, Blessed \emph{be} \emph{thou} of the LORD, my son."],
['Judges','Jdg','17','003','3',"And when he had restored the eleven hundred \emph{shekels} of silver to his mother, his mother said, I had wholly dedicated the silver unto the LORD from my hand for my son, to make a graven image and a molten image: now therefore I will restore it unto thee."],
['Judges','Jdg','17','004','4',"Yet he restored the money unto his mother; and his mother took two hundred \emph{shekels} of silver, and gave them to the founder, who made thereof a graven image and a molten image: and they were in the house of Micah."],
['Judges','Jdg','17','005','5',"And the man Micah had an house of gods, and made an ephod, and teraphim, and consecrated one of his sons, who became his priest."],
['Judges','Jdg','17','006','6',"In those days \emph{there} \emph{was} no king in Israel, \emph{but} every man did \emph{that} \emph{which} \emph{was} right in his own eyes."],
['Judges','Jdg','17','007','7',"And there was a young man out of Beth-lehem-judah of the family of Judah, who \emph{was} a Levite, and he sojourned there."],
['Judges','Jdg','17','008','8',"And the man departed out of the city from Beth-lehem-judah to sojourn where he could find \emph{a} \emph{place}: and he came to mount Ephraim to the house of Micah, as he journeyed."],
['Judges','Jdg','17','009','9',"And Micah said unto him, Whence comest thou? And he said unto him, I \emph{am} a Levite of Beth-lehem-judah, and I go to sojourn where I may find \emph{a} \emph{place}."],
['Judges','Jdg','17','010','10',"And Micah said unto him, Dwell with me, and be unto me a father and a priest, and I will give thee ten \emph{shekels} of silver by the year, and a suit of apparel, and thy victuals. So the Levite went in."],
['Judges','Jdg','17','011','11',"And the Levite was content to dwell with the man; and the young man was unto him as one of his sons."],
['Judges','Jdg','17','012','12',"And Micah consecrated the Levite; and the young man became his priest, and was in the house of Micah."],
['Judges','Jdg','17','013','13',"Then said Micah, Now know I that the LORD will do me good, seeing I have a Levite to \emph{my} priest."]]

Judges18 = [
    ['Judges','Jdg','18','001','1',"In those days \emph{there} \emph{was} no king in Israel: and in those days the tribe of the Danites sought them an inheritance to dwell in; for unto that day \emph{all} \emph{their} inheritance had not fallen unto them among the tribes of Israel."],
['Judges','Jdg','18','002','2',"And the children of Dan sent of their family five men from their coasts, men of valour, from Zorah, and from Eshtaol, to spy out the land, and to search it; and they said unto them, Go, search the land: who when they came to mount Ephraim, to the house of Micah, they lodged there."],
['Judges','Jdg','18','003','3',"When they \emph{were} by the house of Micah, they knew the voice of the young man the Levite: and they turned in thither, and said unto him, Who brought thee hither? and what makest thou in this \emph{place}? and what hast thou here?"],
['Judges','Jdg','18','004','4',"And he said unto them, Thus and thus dealeth Micah with me, and hath hired me, and I am his priest."],
['Judges','Jdg','18','005','5',"And they said unto him, Ask counsel, we pray thee, of God, that we may know whether our way which we go shall be prosperous."],
['Judges','Jdg','18','006','6',"And the priest said unto them, Go in peace: before the LORD \emph{is} your way wherein ye go."],
['Judges','Jdg','18','007','7',"Then the five men departed, and came to Laish, and saw the people that \emph{were} therein, how they dwelt careless, after the manner of the Zidonians, quiet and secure; and \emph{there} \emph{was} no magistrate in the land, that might put \emph{them} to shame in \emph{any} thing; and they \emph{were} far from the Zidonians, and had no business with \emph{any} man."],
['Judges','Jdg','18','008','8',"And they came unto their brethren to Zorah and Eshtaol: and their brethren said unto them, What \emph{say} ye?"],
['Judges','Jdg','18','009','9',"And they said, Arise, that we may go up against them: for we have seen the land, and, behold, it \emph{is} very good: and \emph{are} ye still? be not slothful to go, \emph{and} to enter to possess the land."],
['Judges','Jdg','18','010','10',"When ye go, ye shall come unto a people secure, and to a large land: for God hath given it into your hands; a place where \emph{there} \emph{is} no want of any thing that \emph{is} in the earth."],
['Judges','Jdg','18','011','11',"And there went from thence of the family of the Danites, out of Zorah and out of Eshtaol, six hundred men appointed with weapons of war."],
['Judges','Jdg','18','012','12',"And they went up, and pitched in Kirjath-jearim, in Judah: wherefore they called that place Mahaneh-dan unto this day: behold, \emph{it} \emph{is} behind Kirjath-jearim."],
['Judges','Jdg','18','013','13',"And they passed thence unto mount Ephraim, and came unto the house of Micah."],
['Judges','Jdg','18','014','14',"Then answered the five men that went to spy out the country of Laish, and said unto their brethren, Do ye know that there is in these houses an ephod, and teraphim, and a graven image, and a molten image? now therefore consider what ye have to do."],
['Judges','Jdg','18','015','15',"And they turned thitherward, and came to the house of the young man the Levite, \emph{even} unto the house of Micah, and saluted him."],
['Judges','Jdg','18','016','16',"And the six hundred men appointed with their weapons of war, which \emph{were} of the children of Dan, stood by the entering of the gate."],
['Judges','Jdg','18','017','17',"And the five men that went to spy out the land went up, \emph{and} came in thither, \emph{and} took the graven image, and the ephod, and the teraphim, and the molten image: and the priest stood in the entering of the gate with the six hundred men \emph{that} \emph{were} appointed with weapons of war."],
['Judges','Jdg','18','018','18',"And these went into Micah's house, and fetched the carved image, the ephod, and the teraphim, and the molten image. Then said the priest unto them, What do ye?"],
['Judges','Jdg','18','019','19',"And they said unto him, Hold thy peace, lay thine hand upon thy mouth, and go with us, and be to us a father and a priest: \emph{is} \emph{it} better for thee to be a priest unto the house of one man, or that thou be a priest unto a tribe and a family in Israel?"],
['Judges','Jdg','18','020','20',"And the priest's heart was glad, and he took the ephod, and the teraphim, and the graven image, and went in the midst of the people."],
['Judges','Jdg','18','021','21',"So they turned and departed, and put the little ones and the cattle and the carriage before them."],
['Judges','Jdg','18','022','22',"\emph{And} when they were a good way from the house of Micah, the men that \emph{were} in the houses near to Micah's house were gathered together, and overtook the children of Dan."],
['Judges','Jdg','18','023','23',"And they cried unto the children of Dan. And they turned their faces, and said unto Micah, What aileth thee, that thou comest with such a company?"],
['Judges','Jdg','18','024','24',"And he said, Ye have taken away my gods which I made, and the priest, and ye are gone away: and what have I more? and what \emph{is} this \emph{that} ye say unto me, What aileth thee?"],
['Judges','Jdg','18','025','25',"And the children of Dan said unto him, Let not thy voice be heard among us, lest angry fellows run upon thee, and thou lose thy life, with the lives of thy household."],
['Judges','Jdg','18','026','26',"And the children of Dan went their way: and when Micah saw that they \emph{were} too strong for him, he turned and went back unto his house."],
['Judges','Jdg','18','027','27',"And they took \emph{the} \emph{things} which Micah had made, and the priest which he had, and came unto Laish, unto a people \emph{that} \emph{were} at quiet and secure: and they smote them with the edge of the sword, and burnt the city with fire."],
['Judges','Jdg','18','028','28',"And \emph{there} \emph{was} no deliverer, because it \emph{was} far from Zidon, and they had no business with \emph{any} man; and it was in the valley that \emph{lieth} by Beth-rehob. And they built a city, and dwelt therein."],
['Judges','Jdg','18','029','29',"And they called the name of the city Dan, after the name of Dan their father, who was born unto Israel: howbeit the name of the city \emph{was} Laish at the first."],
['Judges','Jdg','18','030','30',"And the children of Dan set up the graven image: and Jonathan, the son of Gershom, the son of Manasseh, he and his sons were priests to the tribe of Dan until the day of the captivity of the land."],
['Judges','Jdg','18','031','31',"And they set them up Micah's graven image, which he made, all the time that the house of God was in Shiloh."]]


Judges19 = [
    ['Judges','Jdg','19','001','1',"And it came to pass in those days, when \emph{there} \emph{was} no king in Israel, that there was a certain Levite sojourning on the side of mount Ephraim, who took to him a concubine out of Beth-lehem-judah."],
['Judges','Jdg','19','002','2',"And his concubine played the whore against him, and went away from him unto her father's house to Beth-lehem-judah, and was there four whole months."],
['Judges','Jdg','19','003','3',"And her husband arose, and went after her, to speak friendly unto her, \emph{and} to bring her again, having his servant with him, and a couple of asses: and she brought him into her father's house: and when the father of the damsel saw him, he rejoiced to meet him."],
['Judges','Jdg','19','004','4',"And his father in law, the damsel's father, retained him; and he abode with him three days: so they did eat and drink, and lodged there."],
['Judges','Jdg','19','005','5',"And it came to pass on the fourth day, when they arose early in the morning, that he rose up to depart: and the damsel's father said unto his son in law, Comfort thine heart with a morsel of bread, and afterward go your way."],
['Judges','Jdg','19','006','6',"And they sat down, and did eat and drink both of them together: for the damsel's father had said unto the man, Be content, I pray thee, and tarry all night, and let thine heart be merry."],
['Judges','Jdg','19','007','7',"And when the man rose up to depart, his father in law urged him: therefore he lodged there again."],
['Judges','Jdg','19','008','8',"And he arose early in the morning on the fifth day to depart: and the damsel's father said, Comfort thine heart, I pray thee. And they tarried until afternoon, and they did eat both of them."],
['Judges','Jdg','19','009','9',"And when the man rose up to depart, he, and his concubine, and his servant, his father in law, the damsel's father, said unto him, Behold, now the day draweth toward evening, I pray you tarry all night: behold, the day groweth to an end, lodge here, that thine heart may be merry; and to morrow get you early on your way, that thou mayest go home."],
['Judges','Jdg','19','010','10',"But the man would not tarry that night, but he rose up and departed, and came over against Jebus, which \emph{is} Jerusalem; and \emph{there} \emph{were} with him two asses saddled, his concubine also \emph{was} with him."],
['Judges','Jdg','19','011','11',"\emph{And} when they \emph{were} by Jebus, the day was far spent; and the servant said unto his master, Come, I pray thee, and let us turn in into this city of the Jebusites, and lodge in it."],
['Judges','Jdg','19','012','12',"And his master said unto him, We will not turn aside hither into the city of a stranger, that \emph{is} not of the children of Israel; we will pass over to Gibeah."],
['Judges','Jdg','19','013','13',"And he said unto his servant, Come, and let us draw near to one of these places to lodge all night, in Gibeah, or in Ramah."],
['Judges','Jdg','19','014','14',"And they passed on and went their way; and the sun went down upon them \emph{when} \emph{they} \emph{were} by Gibeah, which \emph{belongeth} to Benjamin."],
['Judges','Jdg','19','015','15',"And they turned aside thither, to go in \emph{and} to lodge in Gibeah: and when he went in, he sat him down in a street of the city: for \emph{there} \emph{was} no man that took them into his house to lodging."],
['Judges','Jdg','19','016','16',"And, behold, there came an old man from his work out of the field at even, which \emph{was} also of mount Ephraim; and he sojourned in Gibeah: but the men of the place \emph{were} Benjamites."],
['Judges','Jdg','19','017','17',"And when he had lifted up his eyes, he saw a wayfaring man in the street of the city: and the old man said, Whither goest thou? and whence comest thou?"],
['Judges','Jdg','19','018','18',"And he said unto him, We \emph{are} passing from Beth-lehem-judah toward the side of mount Ephraim; from thence \emph{am} I: and I went to Beth-lehem-judah, but I \emph{am} \emph{now} going to the house of the LORD; and there \emph{is} no man that receiveth me to house."],
['Judges','Jdg','19','019','19',"Yet there is both straw and provender for our asses; and there is bread and wine also for me, and for thy handmaid, and for the young man \emph{which} \emph{is} with thy servants: \emph{there} \emph{is} no want of any thing."],
['Judges','Jdg','19','020','20',"And the old man said, Peace \emph{be} with thee; howsoever \emph{let} all thy wants \emph{lie} upon me; only lodge not in the street."],
['Judges','Jdg','19','021','21',"So he brought him into his house, and gave provender unto the asses: and they washed their feet, and did eat and drink."],
['Judges','Jdg','19','022','22',"\emph{Now} as they were making their hearts merry, behold, the men of the city, certain sons of Belial, beset the house round about, \emph{and} beat at the door, and spake to the master of the house, the old man, saying, Bring forth the man that came into thine house, that we may know him."],
['Judges','Jdg','19','023','23',"And the man, the master of the house, went out unto them, and said unto them, Nay, my brethren, \emph{nay}, I pray you, do not \emph{so} wickedly; seeing that this man is come into mine house, do not this folly."],
['Judges','Jdg','19','024','24',"Behold, \emph{here} \emph{is} my daughter a maiden, and his concubine; them I will bring out now, and humble ye them, and do with them what seemeth good unto you: but unto this man do not so vile a thing."],
['Judges','Jdg','19','025','25',"But the men would not hearken to him: so the man took his concubine, and brought her forth unto them; and they knew her, and abused her all the night until the morning: and when the day began to spring, they let her go."],
['Judges','Jdg','19','026','26',"Then came the woman in the dawning of the day, and fell down at the door of the man's house where her lord \emph{was}, till it was light."],
['Judges','Jdg','19','027','27',"And her lord rose up in the morning, and opened the doors of the house, and went out to go his way: and, behold, the woman his concubine was fallen down \emph{at} the door of the house, and her hands \emph{were} upon the threshold."],
['Judges','Jdg','19','028','28',"And he said unto her, Up, and let us be going. But none answered. Then the man took her \emph{up} upon an ass, and the man rose up, and gat him unto his place."],
['Judges','Jdg','19','029','29',"And when he was come into his house, he took a knife, and laid hold on his concubine, and divided her, \emph{together} with her bones, into twelve pieces, and sent her into all the coasts of Israel."],
['Judges','Jdg','19','030','30',"And it was so, that all that saw it said, There was no such deed done nor seen from the day that the children of Israel came up out of the land of Egypt unto this day: consider of it, take advice, and speak \emph{your} \emph{minds}."]]


Judges20 = [
    ['Judges','Jdg','20','001','1',"Then all the children of Israel went out, and the congregation was gathered together as one man, from Dan even to Beer-sheba, with the land of Gilead, unto the LORD in Mizpeh."],
    ['Judges','Jdg','20','002','2',"And the chief of all the people, \emph{even} of all the tribes of Israel, presented themselves in the assembly of the people of God, four hundred thousand footmen that drew sword."],
    ['Judges','Jdg','20','003','3',"(Now the children of Benjamin heard that the children of Israel were gone up to Mizpeh.) Then said the children of Israel, Tell \emph{us}, how was this wickedness?"],
    ['Judges','Jdg','20','004','4',"And the Levite, the husband of the woman that was slain, answered and said, I came into Gibeah that \emph{belongeth} to Benjamin, I and my concubine, to lodge."],
    ['Judges','Jdg','20','005','5',"And the men of Gibeah rose against me, and beset the house round about upon me by night, \emph{and} thought to have slain me: and my concubine have they forced, that she is dead."],
    ['Judges','Jdg','20','006','6',"And I took my concubine, and cut her in pieces, and sent her throughout all the country of the inheritance of Israel: for they have committed lewdness and folly in Israel."],
    ['Judges','Jdg','20','007','7',"Behold, ye \emph{are} all children of Israel; give here your advice and counsel."],
    ['Judges','Jdg','20','008','8',"And all the people arose as one man, saying, We will not any \emph{of} \emph{us} go to his tent, neither will we any \emph{of} \emph{us} turn into his house."],
    ['Judges','Jdg','20','009','9',"But now this \emph{shall} \emph{be} the thing which we will do to Gibeah; \emph{we} \emph{will} \emph{go} \emph{up} by lot against it;"],
    ['Judges','Jdg','20','010','10',"And we will take ten men of an hundred throughout all the tribes of Israel, and an hundred of a thousand, and a thousand out of ten thousand, to fetch victual for the people, that they may do, when they come to Gibeah of Benjamin, according to all the folly that they have wrought in Israel."],
    ['Judges','Jdg','20','011','11',"So all the men of Israel were gathered against the city, knit together as one man."],
    ['Judges','Jdg','20','012','12',"And the tribes of Israel sent men through all the tribe of Benjamin, saying, What wickedness \emph{is} this that is done among you?"],
    ['Judges','Jdg','20','013','13',"Now therefore deliver \emph{us} the men, the children of Belial, which \emph{are} in Gibeah, that we may put them to death, and put away evil from Israel. But the children of Benjamin would not hearken to the voice of their brethren the children of Israel:"],
    ['Judges','Jdg','20','014','14',"But the children of Benjamin gathered themselves together out of the cities unto Gibeah, to go out to battle against the children of Israel."],
    ['Judges','Jdg','20','015','15',"And the children of Benjamin were numbered at that time out of the cities twenty and six thousand men that drew sword, beside the inhabitants of Gibeah, which were numbered seven hundred chosen men."],
    ['Judges','Jdg','20','016','16',"Among all this people \emph{there} \emph{were} seven hundred chosen men lefthanded; every one could sling stones at an hair \emph{breadth}, and not miss."],
    ['Judges','Jdg','20','017','17',"And the men of Israel, beside Benjamin, were numbered four hundred thousand men that drew sword: all these \emph{were} men of war."],
    ['Judges','Jdg','20','018','18',"And the children of Israel arose, and went up to the house of God, and asked counsel of God, and said, Which of us shall go up first to the battle against the children of Benjamin? And the LORD said, Judah \emph{shall} \emph{go} \emph{up} first."],
    ['Judges','Jdg','20','019','19',"And the children of Israel rose up in the morning, and encamped against Gibeah."],
    ['Judges','Jdg','20','020','20',"And the men of Israel went out to battle against Benjamin; and the men of Israel put themselves in array to fight against them at Gibeah."],
    ['Judges','Jdg','20','021','21',"And the children of Benjamin came forth out of Gibeah, and destroyed down to the ground of the Israelites that day twenty and two thousand men."],
    ['Judges','Jdg','20','022','22',"And the people the men of Israel encouraged themselves, and set their battle again in array in the place where they put themselves in array the first day."],
    ['Judges','Jdg','20','023','23',"(And the children of Israel went up and wept before the LORD until even, and asked counsel of the LORD, saying, Shall I go up again to battle against the children of Benjamin my brother? And the LORD said, Go up against him.)"],
    ['Judges','Jdg','20','024','24',"And the children of Israel came near against the children of Benjamin the second day."],
    ['Judges','Jdg','20','025','25',"And Benjamin went forth against them out of Gibeah the second day, and destroyed down to the ground of the children of Israel again eighteen thousand men; all these drew the sword."],
    ['Judges','Jdg','20','026','26',"Then all the children of Israel, and all the people, went up, and came unto the house of God, and wept, and sat there before the LORD, and fasted that day until even, and offered burnt offerings and peace offerings before the LORD."],
    ['Judges','Jdg','20','027','27',"And the children of Israel enquired of the LORD, (for the ark of the covenant of God \emph{was} there in those days,"],
    ['Judges','Jdg','20','028','28',"And Phinehas, the son of Eleazar, the son of Aaron, stood before it in those days,) saying, Shall I yet again go out to battle against the children of Benjamin my brother, or shall I cease? And the LORD said, Go up; for to morrow I will deliver them into thine hand."],
    ['Judges','Jdg','20','029','29',"And Israel set liers in wait round about Gibeah."],
    ['Judges','Jdg','20','030','30',"And the children of Israel went up against the children of Benjamin on the third day, and put themselves in array against Gibeah, as at other times."],
    ['Judges','Jdg','20','031','31',"And the children of Benjamin went out against the people, \emph{and} were drawn away from the city; and they began to smite of the people, \emph{and} kill, as at other times, in the highways, of which one goeth up to the house of God, and the other to Gibeah in the field, about thirty men of Israel."],
    ['Judges','Jdg','20','032','32',"And the children of Benjamin said, They \emph{are} smitten down before us, as at the first. But the children of Israel said, Let us flee, and draw them from the city unto the highways."],
    ['Judges','Jdg','20','033','33',"And all the men of Israel rose up out of their place, and put themselves in array at Baal-tamar: and the liers in wait of Israel came forth out of their places, \emph{even} out of the meadows of Gibeah."],
    ['Judges','Jdg','20','034','34',"And there came against Gibeah ten thousand chosen men out of all Israel, and the battle was sore: but they knew not that evil \emph{was} near them."],
    ['Judges','Jdg','20','035','35',"And the LORD smote Benjamin before Israel: and the children of Israel destroyed of the Benjamites that day twenty and five thousand and an hundred men: all these drew the sword."],
    ['Judges','Jdg','20','036','36',"So the children of Benjamin saw that they were smitten: for the men of Israel gave place to the Benjamites, because they trusted unto the liers in wait which they had set beside Gibeah."],
    ['Judges','Jdg','20','037','37',"And the liers in wait hasted, and rushed upon Gibeah; and the liers in wait drew \emph{themselves} along, and smote all the city with the edge of the sword."],
    ['Judges','Jdg','20','038','38',"Now there was an appointed sign between the men of Israel and the liers in wait, that they should make a great flame with smoke rise up out of the city."],
    ['Judges','Jdg','20','039','39',"And when the men of Israel retired in the battle, Benjamin began to smite \emph{and} kill of the men of Israel about thirty persons: for they said, Surely they are smitten down before us, as \emph{in} the first battle."],
    ['Judges','Jdg','20','040','40',"But when the flame began to arise up out of the city with a pillar of smoke, the Benjamites looked behind them, and, behold, the flame of the city ascended up to heaven."],
    ['Judges','Jdg','20','041','41',"And when the men of Israel turned again, the men of Benjamin were amazed: for they saw that evil was come upon them."],
    ['Judges','Jdg','20','042','42',"Therefore they turned \emph{their} \emph{backs} before the men of Israel unto the way of the wilderness; but the battle overtook them; and them which \emph{came} out of the cities they destroyed in the midst of them."],
    ['Judges','Jdg','20','043','43',"\emph{Thus} they inclosed the Benjamites round about, \emph{and} chased them, \emph{and} trode them down with ease over against Gibeah toward the sunrising."],
    ['Judges','Jdg','20','044','44',"And there fell of Benjamin eighteen thousand men; all these \emph{were} men of valour."],
    ['Judges','Jdg','20','045','45',"And they turned and fled toward the wilderness unto the rock of Rimmon: and they gleaned of them in the highways five thousand men; and pursued hard after them unto Gidom, and slew two thousand men of them."],
    ['Judges','Jdg','20','046','46',"So that all which fell that day of Benjamin were twenty and five thousand men that drew the sword; all these \emph{were} men of valour."],
    ['Judges','Jdg','20','047','47',"But six hundred men turned and fled to the wilderness unto the rock Rimmon, and abode in the rock Rimmon four months."],
    ['Judges','Jdg','20','048','48',"And the men of Israel turned again upon the children of Benjamin, and smote them with the edge of the sword, as well the men of \emph{every} city, as the beast, and all that came to hand: also they set on fire all the cities that they came to."]
]


Judges21 = [
    ['Judges','Jdg','21','001','1',"Now the men of Israel had sworn in Mizpeh, saying, There shall not any of us give his daughter unto Benjamin to wife."],
['Judges','Jdg','21','002','2',"And the people came to the house of God, and abode there till even before God, and lifted up their voices, and wept sore;"],
['Judges','Jdg','21','003','3',"And said, O LORD God of Israel, why is this come to pass in Israel, that there should be to day one tribe lacking in Israel?"],
['Judges','Jdg','21','004','4',"And it came to pass on the morrow, that the people rose early, and built there an altar, and offered burnt offerings and peace offerings."],
['Judges','Jdg','21','005','5',"And the children of Israel said, Who \emph{is} \emph{there} among all the tribes of Israel that came not up with the congregation unto the LORD? For they had made a great oath concerning him that came not up to the LORD to Mizpeh, saying, He shall surely be put to death."],
['Judges','Jdg','21','006','6',"And the children of Israel repented them for Benjamin their brother, and said, There is one tribe cut off from Israel this day."],
['Judges','Jdg','21','007','7',"How shall we do for wives for them that remain, seeing we have sworn by the LORD that we will not give them of our daughters to wives?"],
['Judges','Jdg','21','008','8',"And they said, What one \emph{is} \emph{there} of the tribes of Israel that came not up to Mizpeh to the LORD? And, behold, there came none to the camp from Jabesh-gilead to the assembly."],
['Judges','Jdg','21','009','9',"For the people were numbered, and, behold, \emph{there} \emph{were} none of the inhabitants of Jabesh-gilead there."],
['Judges','Jdg','21','010','10',"And the congregation sent thither twelve thousand men of the valiantest, and commanded them, saying, Go and smite the inhabitants of Jabesh-gilead with the edge of the sword, with the women and the children."],
['Judges','Jdg','21','011','11',"And this \emph{is} the thing that ye shall do, Ye shall utterly destroy every male, and every woman that hath lain by man."],
['Judges','Jdg','21','012','12',"And they found among the inhabitants of Jabesh-gilead four hundred young virgins, that had known no man by lying with any male: and they brought them unto the camp to Shiloh, which \emph{is} in the land of Canaan."],
['Judges','Jdg','21','013','13',"And the whole congregation sent \emph{some} to speak to the children of Benjamin that \emph{were} in the rock Rimmon, and to call peaceably unto them."],
['Judges','Jdg','21','014','14',"And Benjamin came again at that time; and they gave them wives which they had saved alive of the women of Jabesh-gilead: and yet so they sufficed them not."],
['Judges','Jdg','21','015','15',"And the people repented them for Benjamin, because that the LORD had made a breach in the tribes of Israel."],
['Judges','Jdg','21','016','16',"Then the elders of the congregation said, How shall we do for wives for them that remain, seeing the women are destroyed out of Benjamin?"],
['Judges','Jdg','21','017','17',"And they said, \emph{There} \emph{must} \emph{be} an inheritance for them that be escaped of Benjamin, that a tribe be not destroyed out of Israel."],
['Judges','Jdg','21','018','18',"Howbeit we may not give them wives of our daughters: for the children of Israel have sworn, saying, Cursed \emph{be} he that giveth a wife to Benjamin."],
['Judges','Jdg','21','019','19',"Then they said, Behold, \emph{there} \emph{is} a feast of the LORD in Shiloh yearly \emph{in} \emph{a} \emph{place} which \emph{is} on the north side of Beth-el, on the east side of the highway that goeth up from Beth-el to Shechem, and on the south of Lebonah."],
['Judges','Jdg','21','020','20',"Therefore they commanded the children of Benjamin, saying, Go and lie in wait in the vineyards;"],
['Judges','Jdg','21','021','21',"And see, and, behold, if the daughters of Shiloh come out to dance in dances, then come ye out of the vineyards, and catch you every man his wife of the daughters of Shiloh, and go to the land of Benjamin."],
['Judges','Jdg','21','022','22',"And it shall be, when their fathers or their brethren come unto us to complain, that we will say unto them, Be favourable unto them for our sakes: because we reserved not to each man his wife in the war: for ye did not give unto them at this time, \emph{that} ye should be guilty."],
['Judges','Jdg','21','023','23',"And the children of Benjamin did so, and took \emph{them} wives, according to their number, of them that danced, whom they caught: and they went and returned unto their inheritance, and repaired the cities, and dwelt in them."],
['Judges','Jdg','21','024','24',"And the children of Israel departed thence at that time, every man to his tribe and to his family, and they went out from thence every man to his inheritance."],
['Judges','Jdg','21','025','25',"In those days \emph{there} \emph{was} no king in Israel: every man did \emph{that} \emph{which} \emph{was} right in his own eyes."]
]



def main():
    ProcessVerses('Judges21',Judges21)

if __name__ == "__main__":
    main()




%%%%%%%%%%%%%%%%%%%%%%%%%%%
Chapter with Latex format
%%%%%%%%%%%%%%%%%%%%%%%%%%%


\footnote{\textcolor[cmyk]{0.99998,1,0,0}{\hyperlink{TOC}{Return to end of Table of Contents.}}}\textcolor[cmyk]{0.99998,1,0,0}{Now the men of Israel had sworn in Mizpeh, saying, There shall not any of us give his daughter unto Benjamin to wife.}
[2] \textcolor[cmyk]{0.99998,1,0,0}{And the people came to the house of God, and abode there till even before God, and lifted up their voices, and wept sore;}
[3] \textcolor[cmyk]{0.99998,1,0,0}{And said, O LORD God of Israel, why is this come to pass in Israel, that there should be to day one tribe lacking in Israel?}
[4] \textcolor[cmyk]{0.99998,1,0,0}{And it came to pass on the morrow, that the people rose early, and built there an altar, and offered burnt offerings and peace offerings.}
[5] \textcolor[cmyk]{0.99998,1,0,0}{And the children of Israel said, Who \emph{is} \emph{there} among all the tribes of Israel that came not up with the congregation unto 

dances & 1 \\ \hline
then & 1 \\ \hline
catch & 1 \\ \hline
you & 1 \\ \hline
go & 1 \\ \hline
when & 1 \\ \hline
fathers & 1 \\ \hline
or & 1 \\ \hline
brethren & 1 \\ \hline
complain & 1 \\ \hline
say & 1 \\ \hline
Be & 1 \\ \hline
favourable & 1 \\ \hline
sakes & 1 \\ \hline
reserved & 1 \\ \hline
each & 1 \\ \hline
war & 1 \\ \hline
guilty & 1 \\ \hline
took & 1 \\ \hline
\emph{them} & 1 \\ \hline
according & 1 \\ \hline
number & 1 \\ \hline
danced & 1 \\ \hline
whom & 1 \\ \hline
caught & 1 \\ \hline
returned & 1 \\ \hline
repaired & 1 \\ \hline
cities & 1 \\ \hline
dwelt & 1 \\ \hline
departed & 1 \\ \hline
family & 1 \\ \hline
In & 1 \\ \hline
those & 1 \\ \hline
days & 1 \\ \hline
king & 1 \\ \hline
\emph{which} & 1 \\ \hline
right & 1 \\ \hline
own & 1 \\ \hline
eyes & 1 \\ \hline
\end{longtable}
\end{center}



\normalsize



\subsection{Words Alphabetically}
\scriptsize
\begin{center}
\begin{longtable}{l|r}
\caption[Word Alphabetically in Judges 21]{Word Alphabetically in Ju